# objectif du notebook
- création les jeux de données à annoter manuellement sur excel
    - distribution aléatoire entre N annotateurs
- lecture des jeux de données annotés sur excel
- creation d'un dictionnaire pour les données de références (DR): [clé=nom_theme, valeur=list(code_theme)]
    - ex: 'livre-jeu': ['YBCS2', 'YBG']
        ici: pour le même nom_theme, chaque evaluateur a donné un code Thema différent

# imports librairies

In [2]:
import os 
import random
import itertools
import uuid
import pandas as pd
import sklearn.metrics
import sys
sys.path.append('../')
sys.path.append('../andre')
import andre.utils as utils
import copy
import json

# création les jeux de données à annoter manuellement sur excel

## chargement des données et prétraitement

In [9]:
ecole_des_loisirs = ['âne',
 'animaux des champs et des bois',
 'animaux divers',
 'animaux domestiques',
 'araignée',
 'autruche, émeu',
 'baleine',
 'belette',
 'bison',
 'blaireau',
 'caméléon',
 'canard, oie',
 'castor',
 'cervidés',
 'chameau, dromadaire',
 'chamois',
 'chat',
 'chauve souris',
 'cheval',
 'chien',
 'cochon',
 'coyote, chacal',
 'crabe',
 'crocodile, alligator',
 'dauphin',
 'des animaux et des hommes',
 'dindon',
 'dinosaure (animal)',
 'écureuil',
 'éléphant',
 'escargot',
 'félins divers',
 'girafe',
 'gnou',
 'grenouille, crapaud, têtard',
 'hamster, cobaye',
 'hérisson, porc-épic',
 'hibou',
 'hippocampe',
 'hippopotame',
 'insectes divers',
 'kangourou',
 'koala',
 'lamantin',
 'lapin, lièvre',
 'lézard',
 'lion',
 'loir, lérot',
 'loup',
 'loutre',
 'lynx',
 'mammouth',
 'marmotte',
 'morse',
 'mouton, chèvre, bouc',
 'oiseaux divers',
 'opossum',
 'ornithorynque',
 'orque',
 'ours',
 'panda',
 'papillon',
 'paresseux (animal)',
 'phoque',
 'pieuvre, poulpe',
 'pingouin, manchot',
 'poissons divers',
 'poule, poussin, coq',
 'raton laveur',
 'renard',
 'requin',
 'rhinocéros',
 'sanglier',
 'serpent',
 'singe, chimpanzé, gorille',
 'souris, mulot, rat, campagnol',
 'suricate, mangouste',
 'tatou',
 'taupe',
 'tigre',
 'tortue',
 'vache, veau, taureau',
 'ver de terre, lombric',
 'yéti',
 'zèbre',
 'art pariétal, peinture rupestre',
 'art traditionnel, arts premiers',
 'cinéma',
 'cirque',
 'clown',
 "commedia dell'arte",
 'créativité, création artistique',
 'danse',
 'funambule',
 'guignol',
 'Littérature, livre, écrivain',
 'marionnette',
 'mime',
 'musique',
 'opéra',
 'peinture',
 'photographie',
 'photographie (livre de)',
 'sculpture',
 'street-art',
 'théâtre (activité)',
 'dent, dent de lait',
 'main',
 'schéma corporel',
 'squelette',
 'hygiène, propreté',
 'pipi, caca, toilettes',
 'pot',
 'poux',
 'beauté, laideur',
 'coquetterie',
 'imagerie du corps',
 'autisme',
 'corps et ses différences (le)',
 'handicap',
 'lunettes',
 'grossesse, avortement',
 'naissance, maternité',
 'accident',
 'alcoolisme',
 'anorexie',
 'appétit',
 'bobo',
 'boulimie',
 'drogue, toxicomanie',
 'épidémie',
 'hospitalisation',
 'maladie, docteur',
 'pipi au lit, énurésie',
 'santé',
 'sida',
 'homosexualité',
 'puberté, transformation du corps',
 'sexualité',
 'dormir, coucher',
 'peur du noir, cauchemar',
 'rêve',
 'les cinq sens',
 'prévention (le corps)',
 'repère spatio temporel',
 'taille',
 "apprentissage de l'autonomie",
 'apprentissage de la vie',
 'premiers apprentissages',
 'bébé',
 'crèche',
 'tétine, biberon',
 'apprentissage des langues',
 'classe de nature',
 'collège, lycée, professeur',
 'école buissonnière',
 'école, instituteur',
 'pensionnat',
 'rentrée des classes',
 'sortie scolaire',
 'université',
 'mort, deuil',
 'prise de responsabilité',
 'temps (qui passe)',
 'vieillesse',
 'généalogie',
 'recherche de ses origines',
 'bêtises, désobéissance',
 'conflit parents, enfants',
 'fratrie',
 'fugue',
 'grand-mère',
 'grand-père',
 'grands-parents',
 'homoparentalité',
 'jumeaux, jumelles',
 'naissance, nouveau bébé dans la fratrie',
 'relation avec la mère',
 'relation avec le père',
 'relation enfant, adulte',
 'sœurs et frères : aîné, cadet',
 'déménagement',
 'désordre',
 'vie quotidienne',
 'adoption',
 'dispute',
 'divorce, séparation',
 'en famille',
 'enfant abandonné, abandon',
 'enfant maltraité',
 "famille d'accueil",
 'famille éclatée',
 'famille monoparentale',
 'famille recomposée, remariage',
 'la famille',
 'mariage',
 'orphelin, orpheline',
 'secrets de famille',
 'vacances en famille',
 'aventure et mystère',
 "cape et d'épée",
 'histoire policière',
 'récit de voyage',
 "roman d'aventure",
 'conte détourné',
 'conte détourné, le bal des 12 princesses',
 'conte détourné, barbe bleue',
 'conte détourné, blanche-neige',
 "conte détourné, boucle d'or et les 3 ours",
 'conte détourné, cendrillon',
 'conte détourné, la belle au bois dormant',
 'conte détourné, la princesse au petit pois',
 'conte détourné, le loup et les 7 cabris',
 'conte détourné, le petit chaperon rouge',
 'conte détourné, le petit poucet',
 'conte détourné, les 3 petits cochons',
 'conte détourné, les fées',
 'contes du monde entier',
 'contes et recueil de contes',
 'fables',
 'classique abrégé',
 'classique intégral',
 "classiques du livre d'images",
 'fantastique',
 'fantasy',
 'science-fiction',
 'fables, fabliaux',
 'Littérature du Moyen Âge',
 'roman de la table ronde',
 'cow-boy',
 'super-héros',
 'ange',
 'chevalier',
 'diable',
 'dragon',
 'fantôme',
 'fée',
 'géant',
 'génie',
 'licorne',
 'lutin, troll, elfe, gnome',
 'magicien, magie',
 'monstre',
 'nain',
 'ogre',
 'personnage de conte (divers)',
 'prince, princesse',
 'roi, reine',
 'sirène',
 'sorcier, sorcière',
 'vampire',
 'sketch',
 'théâtre classique',
 'théâtre contemporain',
 'théâtre, pièce de théâtre',
 'atlas',
 'autobiographie',
 'aventure',
 'bande dessinée',
 'biographie',
 'comptine',
 'conte',
 'énigmes',
 'épouvante',
 'extraterrestre',
 'humour',
 'journal',
 'lettres, correspondance',
 'livre dont on est le héros',
 'Livre ukrainien',
 'Livre-jeu',
 'MOOC',
 'mythologie',
 'parodie, pastiche',
 'philosophie',
 'poésie',
 'proverbe',
 "recueil d'histoires",
 'recueil de nouvelles',
 'rime',
 "roman de cap et d'épée",
 'texte fondateur',
 'Afrique (continent africain)',
 'Afrique du nord',
 'Afrique du sud',
 'Algérie',
 'Egypte',
 'Kenya',
 'Libye',
 'Madagascar',
 'Maghreb',
 'Mali',
 'Maroc',
 'Seychelles (îles des)',
 'Soudan',
 'Tanzanie',
 'Tunisie',
 'Zanzibar',
 'Antilles, Caraïbes',
 'Costa-Rica',
 'Cuba',
 'Guatemala',
 'Haïti',
 'Mexique',
 'Porto-Rico',
 'Alaska',
 'Canada',
 'Etats-Unis',
 'Argentine',
 'Bolivie',
 'Brésil',
 'Chili',
 'Colombie',
 'Equateur',
 'Pérou',
 'Vénézuela',
 'Afghanistan',
 'Arabie',
 'Arménie',
 'Asie (en général)',
 'Bahrein',
 'Bangladesh',
 'Birmanie',
 'Bornéo',
 'Cambodge',
 'Chine',
 'Corée',
 'Emirats',
 'Extrème-Orient',
 'Inde',
 'Indonésie',
 'Irak',
 'Iran',
 'Israël, palestine',
 'Japon',
 'Jordanie',
 'Kirghizistan',
 'Koweït',
 'Laos',
 'Liban',
 'Malaisie',
 'Mongolie',
 'Moyen-Orient',
 'Népal',
 'Oman',
 'Pakistan',
 'Philippines',
 'Sri lanka',
 'Syrie',
 'Taïwan',
 'Thaïlande',
 'Tibet',
 'Viêtnam',
 'Yémen',
 'Géorgie',
 'Ouzbékistan',
 'Russie',
 'Turquie',
 'URSS',
 'Esquimau, Inuit',
 'Gaulois',
 'Indien',
 "Indiens d'Amérique",
 'Mayas',
 'Monde arabe',
 'Touareg',
 'Vikings',
 'Allemagne',
 'Autriche',
 'Balkans',
 'Belgique',
 'Danemark',
 'Espagne',
 'Europe',
 'Finlande',
 'France',
 'Grande-Bretagne',
 'Grèce',
 'Hongrie',
 'Islande',
 'Italie',
 'Norvège',
 'Pays nordiques',
 'Pays-Bas',
 'Pologne',
 'Portugal',
 'Roumanie',
 'Suède',
 'Suisse',
 'Tchécoslovaquie',
 'Ukraine',
 'Amazonie',
 'banquise',
 'désert',
 'forêt',
 'île',
 'jungle, forêt lointaine',
 'montagne',
 'rivière, fleuve, étang',
 'Sahara',
 'volcan',
 'Australie',
 'Nouvelle-Calédonie',
 'Nouvelle-Guinée',
 'Nouvelle-Zélande',
 'Polynésie',
 'Arctique, Antarctique',
 'camp de concentration',
 'Commune de paris',
 'conquête',
 'déportation',
 'entre deux guerres',
 'fascisme',
 'génocide arménien',
 'génocide khmère',
 'guerre, conflit',
 'Hiroshima',
 'mafia',
 'nazisme',
 'Première Guerre mondiale',
 'révolution',
 'Révolution française',
 'Seconde Guerre mondiale',
 'terrorisme, guerre civile',
 'Antiquité',
 "Antiquité : l'Egypte",
 'Antiquité : la Gaule',
 'Antiquité : la Grèce',
 'Antiquité : Rome',
 'Gaule (la)',
 "Histoire (l'an 1000)",
 'Histoire : 13e siècle',
 'Histoire : 15e siècle',
 'Histoire : 16e siècle',
 'Histoire : 17e siècle',
 'Histoire : 18e siècle',
 'Histoire : 19e siècle',
 'Histoire : 20e siècle',
 'Histoire : 21e siècle',
 'Histoire : 5e siècle',
 'Histoire : Moyen Âge',
 'Préhistoire',
 'Martin Luther King',
 'château, château fort',
 'Cro-Magnon (Homme de)',
 'dinosaure',
 'Histoire du monde',
 'Mésopotamie',
 'personnage historique',
 'pionniers',
 'bruits et cris',
 'chant, chanson',
 'contraire',
 'couleurs',
 'étymologie',
 'histoire sans texte',
 'images pour jouer',
 'imagier',
 'langage, jeux de mots',
 'livre à compter, chiffres',
 'métamorphose (image)',
 'balle, ballon',
 'cache-cache',
 'cerf-volant',
 'course',
 'déguisement',
 'doudou',
 'jeu',
 'jouets',
 'masque',
 'ours en peluche',
 'poupée',
 'robot',
 'abécédaire',
 'almanach',
 'coloriage, livre à colorier',
 'jeu de rôle',
 'jeu vidéo',
 'les échecs',
 'livre animé',
 'livre de bain',
 'livre de naissance',
 'livre puzzle',
 'chef, meneur, leader',
 'courageux',
 'cupide',
 'curiosité',
 'égoïste',
 'fantaisie, fantaisiste',
 'farceur, farceuse',
 'ingénieux, débrouillard',
 'paresseux',
 'rabat-joie, grincheux',
 'responsable',
 'rêveur, poète',
 'rusé, astucieux',
 'vantard',
 'ami imaginaire',
 'amitié',
 'amour',
 'caprices, entêtement',
 'colère',
 'éco-anxiété',
 'émotion',
 'ennui',
 'gratitude',
 'harcèlement',
 'honte, gêne',
 'humeur',
 'jalousie',
 'méchanceté, cruauté',
 'mensonge',
 'moquerie',
 'nostalgie',
 'peur',
 'regard des autres',
 'rivalité, bagarre, dispute',
 "sentiment d'injustice",
 'sentiment de culpabilité',
 'tendresse, baiser, câlin',
 'timidité',
 'tristesse, chagrin',
 'vengeance',
 'affirmation de soi',
 'genre',
 'imagination, fabulation',
 'initiation, épreuve',
 "le complexe d'Oedipe",
 'portrait',
 "recherche d'absolu",
 "recherche d'identité",
 'secret',
 'souvenir, mémoire',
 'télépathie',
 'thérapie',
 'transgenre',
 'transmission du savoir',
 'hibernation',
 'instinct, education',
 'nourriture, qui mange qui ?',
 'reproduction animale',
 'trace',
 'zoologie',
 'arbre',
 'champignons',
 'fleur',
 'fruit',
 'légume',
 'catastrophe naturelle',
 'intempérie',
 'météo',
 'neige',
 'nuage',
 'pluie, orage',
 'soleil',
 'vent',
 'automne',
 'cycle des saisons',
 'été',
 'heure',
 'hiver',
 'printemps',
 'semaine',
 'biologie',
 'déchets',
 'eau',
 'écologie, environnement',
 'espace, air',
 'feu',
 'métamorphose (biologie)',
 'naturaliste',
 'nuit (dans la)',
 'pierre, caillou, minéral',
 'planète terre',
 "protection de l'environnement",
 'reproduction végétale',
 'vie sous-marine',
 'vie souterraine',
 'astronomie',
 'aventure aérienne et spatiale',
 'ciel',
 'conquête spatiale',
 'cosmonaute, spationaute',
 'étoile',
 'lune',
 'classification des espèces',
 'imprimerie',
 'savant(e), femme/homme de science, inventeur',
 'technologie',
 'avion',
 'bateau',
 'camion, voiture',
 'code de la route',
 'engins divers',
 'métro',
 'montgolfière',
 'moyen de transport',
 'port',
 'train',
 'archéologie',
 'architecture',
 'clonage humain',
 'informatique, internet',
 'physique, chimie',
 'sismologie',
 'spéléologie',
 'transformation des produits naturels',
 'internet',
 'médias',
 'publicité',
 'réseaux sociaux',
 'télévision',
 'conditions sociales',
 'pauvreté, précarité',
 'travail (conditions de)',
 'sectes',
 'sorcellerie',
 'Bohémien, Gitan',
 'bonbon',
 'chapeau',
 'cuisine',
 'culture, tradition',
 'gourmandise',
 'nourriture',
 'vêtement',
 'comportements sociaux',
 'anniversaire et fête',
 'carnaval',
 'fête foraine',
 'galette des rois',
 'halloween',
 'Noël',
 'Noël (sapin de)',
 'Pâques',
 'père noël',
 "pique-nique, goûter d'enfants",
 'bricolage',
 'camping',
 'jeux de rôles',
 'jeux vidéo',
 'livre, lecture',
 'manège',
 'scoutisme',
 'esclavage',
 'exil',
 'immigration, migration',
 'intégration',
 'réfugié, sans-papier',
 'apartheid',
 'communisme',
 'conscience politique',
 'démocratie',
 'dictature',
 'Khmers rouges',
 'totalitarisme',
 'arche de noé',
 'bible',
 'foi',
 'intégrisme',
 'juive (culture)',
 'religions, croyances',
 'arts martiaux',
 'boxe',
 'équitation',
 'exploit sportif',
 'football',
 'Jeux olympiques',
 'natation',
 'patin à glace',
 'roller',
 'rugby',
 'ski',
 'sport',
 'surf',
 'tennis',
 'vélo',
 'yoga',
 'inceste',
 'interdit',
 'justice',
 'maltraitance',
 'prison',
 'violence',
 'violence sexuelle',
 'voleur',
 'argent (système économique)',
 'argent de poche',
 'baby-sitting',
 'boulanger, pâtissier',
 'casting',
 'chômage',
 'coiffeur, coiffure',
 'détective',
 'docteur, médecin',
 'écrivain, écriture',
 'explorateur',
 'facteur',
 'journalisme',
 'juge',
 'métiers divers',
 'péniche, marinier',
 'petits boulots',
 'pompier',
 'travail',
 'travail des enfants',
 'vétérinaire',
 'colonie de vacances, centre de loisirs',
 'tour du monde',
 'vacances',
 "vacances à l'étranger",
 'vacances à la campagne',
 'vacances à la mer',
 'vacances à la montagne',
 'voyage',
 'argent',
 'citoyenneté',
 'communication',
 'confinement',
 'piscine',
 'pouvoir',
 'transhumanisme',
 'campagne',
 'ferme',
 'pêche',
 'retour à la nature',
 'vie rurale',
 'mer',
 'naufrage',
 'pirate, corsaire',
 'plage, bord de mer',
 'conditions de vie dans les villes',
 'jardin public',
 'magasin',
 'marché',
 'musée',
 'poste',
 'rue',
 'urbanisme',
 'ville (la)',
 'zoo',
 'cabane',
 'jardin',
 'lieux et territoires, campagne',
 'maison, habitation']

In [10]:
ecole_des_loisirs_pretraite = [word.lower() for word in ecole_des_loisirs]

In [11]:
ecole_des_loisirs_pretraite

['âne',
 'animaux des champs et des bois',
 'animaux divers',
 'animaux domestiques',
 'araignée',
 'autruche, émeu',
 'baleine',
 'belette',
 'bison',
 'blaireau',
 'caméléon',
 'canard, oie',
 'castor',
 'cervidés',
 'chameau, dromadaire',
 'chamois',
 'chat',
 'chauve souris',
 'cheval',
 'chien',
 'cochon',
 'coyote, chacal',
 'crabe',
 'crocodile, alligator',
 'dauphin',
 'des animaux et des hommes',
 'dindon',
 'dinosaure (animal)',
 'écureuil',
 'éléphant',
 'escargot',
 'félins divers',
 'girafe',
 'gnou',
 'grenouille, crapaud, têtard',
 'hamster, cobaye',
 'hérisson, porc-épic',
 'hibou',
 'hippocampe',
 'hippopotame',
 'insectes divers',
 'kangourou',
 'koala',
 'lamantin',
 'lapin, lièvre',
 'lézard',
 'lion',
 'loir, lérot',
 'loup',
 'loutre',
 'lynx',
 'mammouth',
 'marmotte',
 'morse',
 'mouton, chèvre, bouc',
 'oiseaux divers',
 'opossum',
 'ornithorynque',
 'orque',
 'ours',
 'panda',
 'papillon',
 'paresseux (animal)',
 'phoque',
 'pieuvre, poulpe',
 'pingouin, manch

In [12]:
len(ecole_des_loisirs_pretraite)

744

In [13]:
ricochet = ["Abandon/Sentiment d'abandon",
 'Abécédaire/Alphabet',
 'Abeille',
 'Aborigène',
 'Absence',
 'Absurde',
 'Abus sexuel',
 'Accident',
 'Activités manuelles',
 "Activités/Cahier d'activités",
 "Adaptation (d'œuvre littéraire)",
 'Adolescence',
 'Adoption',
 'Adultère',
 'Affection/Tendresse',
 'Affirmation de soi',
 'Afghanistan',
 'Afrique',
 'Afrique du Nord',
 'Afrique du Sud',
 'Aide humanitaire',
 'Alaska',
 'Album',
 'Album sans texte',
 'Alcool/Alcoolisme',
 'Algérie',
 'Alimentation/Goût',
 'Allemagne',
 'Alphabétisation',
 'Alpinisme',
 'Amérique du Nord',
 'Amérique latine',
 'Amitié',
 'Amnésie',
 'Amour',
 'Analyse de texte',
 'Ane',
 'Ange',
 'Animaux',
 'Anniversaire/Fête',
 'Anorexie/Boulimie',
 'Antilles/Caraïbes',
 'Antiquité (Gaule)',
 'Antiquité (Grèce)',
 'Antiquité (Rome)',
 'Antisémitisme',
 'Apartheid',
 'Apprentissage',
 'Apprentissage de la vie',
 'Araignée',
 'Arbre',
 'Archéologie',
 'Architecture',
 'Argent/Argent de poche',
 'Argentine',
 'Arménie',
 'Art',
 'Art brut',
 'Art urbain',
 'Asie',
 'Astrologie',
 'Astronomie',
 "Atelier d'écriture",
 'Australie',
 'Autisme',
 'Autobiographie',
 'Automne',
 'Automobile/Voiture',
 'Autriche',
 'Autruche',
 'Aventure',
 'Avion/Aviation',
 'Bain/Toilette',
 'Bande dessinée',
 'Bande dessinée (Histoire)',
 'Bandit/Brigand',
 'Banlieue',
 'Bateau/Voilier',
 'Beauté/Laideur',
 'Bébé',
 'Belgique',
 'Berceuse',
 'Bestiaire',
 'Bêtise/Désobéissance',
 'Bhoutan',
 'Bible',
 'Bibliographie/Sélection de livre',
 'Bibliothèque',
 'Bibliothèque jeunesse',
 'Bicyclette/Vélo',
 'Bien-être animal',
 'Bijou/Diamant',
 'Biographie',
 'Bise/Bisou/Baiser',
 'Bison',
 'Blaireau',
 'Bolivie',
 'Bonheur',
 'Botanique',
 'Bouddhisme',
 'Boxe',
 'Brésil',
 'Bretagne',
 'Bruits/Cris',
 'Bulgarie',
 'Burundi',
 'Caillou/Rocher',
 'Calligraphie',
 'Cambodge',
 'Cameroun',
 'Campagne/Ferme',
 'Camping',
 'Canada',
 'Canard/Oie',
 'Caprice/Entêtement',
 'Carnaval',
 'Castor',
 'Catastrophe naturelle',
 'Catastrophe nucléaire',
 'Cécité',
 'Célébrité',
 'Censure',
 'Cerf-volant',
 'Cerf/Biche',
 'Chameau/Dromadaire',
 'Champignon',
 'Chanson/Chant',
 'Chapeau',
 'Chat',
 'Château',
 'Chaud/Froid',
 'Chauve-souris',
 'Chenille',
 'Cheval/Equitation',
 'Chevalier/Chevalerie',
 'Chien',
 'Chiffre/Nombre/Livre à compter',
 'Chimie',
 'Chine',
 'Chocolat',
 'Chômage',
 'Chouette/Hibou',
 'Ciel',
 'Cinéma/Vidéo',
 'Cinq sens',
 'Cirque',
 'Citation',
 'Citoyenneté/Civisme',
 'Clown',
 'Coccinelle',
 'Cochon/Sanglier',
 'Coiffeur',
 'Colère',
 'Collection/Collectionneur',
 'Collège',
 'Colombie',
 'Colonie de vacances',
 'Colonisation/Colonie',
 'Coloriage',
 'Comédie musicale',
 'Communication',
 'Compétition',
 'Comptine',
 'Concentration',
 'Condamnation à mort',
 'Condition féminine',
 'Condition sociale',
 'Conformisme',
 'Congo',
 "Conquête de l'Ouest américain",
 'Consommation',
 'Conte',
 'Conte (adaptation)',
 'Contraire',
 'Corée',
 'Corps',
 'Correspondance',
 'Corrida',
 'Corse',
 'Couleur',
 'Courage',
 'Crocodile/Alligator',
 'Cuba',
 'Cuisine/Recette',
 'Culpabilité',
 'Curiosité',
 'Cygne',
 'Danemark',
 'Danse/Ballet',
 'Dauphin/Baleine',
 'Débarquement',
 'Découverte',
 'Déguisement',
 'Délinquance',
 'Déménagement',
 'Dent',
 'Dépression',
 'Désert',
 'Désespoir',
 'Design',
 'Dessin',
 'Dessin animé',
 'Détente/Relaxation',
 'Développement durable',
 'Devinette',
 'Diable',
 'Dictature',
 'Différence',
 'Dinosaure',
 'Disparition',
 'Documentaire',
 'Dormir/Coucher',
 'Double',
 'Doudou/Peluche',
 'Dragon',
 'Drogue',
 'Droit',
 "Droits de l'enfant",
 "Droits de l'homme",
 'Dyslexie/Trouble de la lecture',
 'Dystopie',
 'Eau',
 'Ecole',
 'Ecole maternelle',
 'Ecole primaire',
 'Ecologie',
 'Economie/Finance',
 'Ecosse',
 'Ecriture',
 'Ecureuil',
 'Edition',
 'Edition - Petite Edition',
 'Education',
 'Egalité',
 'Egoïsme',
 'Egypte',
 'Egypte ancienne',
 'Elan',
 'Electricité',
 'Eléphant',
 'Emotions',
 'Encyclopédie/Dictionnaire',
 'Energie',
 'Enfance',
 'Enfant des rues',
 'Enfant sauvage',
 'Enigme',
 'Enlèvement/Kidnapping',
 'Ennui',
 'Enquête',
 'Entraide/Solidarité',
 'Environnement',
 'Epreuve/Initiation',
 'Équateur',
 'Escargot',
 'Esclavage',
 'Espace/vaisseaux spatiaux',
 'Espagne',
 'Espionnage/Espion',
 'Espoir',
 'Etats-Unis',
 'Eté',
 'Ethiopie',
 'Ethique/Morale',
 'Etoile',
 'Europe',
 "Europe de l'Est",
 'Excision',
 'Exclusion',
 'Exil',
 'Exploit',
 'Exposition',
 'Extraterrestre',
 'Fable',
 'Famille',
 "Famille d'accueil",
 'Famille recomposée',
 'Fantaisie',
 'Fantastique',
 'Fantôme',
 'Fascisme',
 'Fée',
 'Femme/Homme',
 'Fête foraine',
 'Feu/Incendie',
 'Fille/Garçon',
 'Finlande',
 'Fleur',
 'Football',
 'Forêt',
 'Forme',
 'Fourmi',
 'France',
 'Fruit',
 'Fugue',
 'Généalogie/Arbre généalogique',
 'Générosité',
 'Géographie/Atlas',
 'Girafe',
 'Gitan/Bohémien',
 'Gourmandise',
 'Grand Nord',
 'Grand-Mère',
 'Grand-Père',
 'Grand/Petit',
 'Grande-Bretagne',
 'Grèce',
 'Grenouille/Crapaud',
 'Grève',
 'Grimace',
 'Gros/Maigre',
 'Guatemala',
 'Guerre/Conflit',
 'Guide/Répertoire',
 'Guyane',
 'Habitation/Maison',
 'Halloween',
 "Hamster/Cochon d'Inde",
 'Handicap',
 'Harcèlement',
 'Hérisson',
 'Héros',
 'Heure',
 'Hippopotame',
 'Histoire',
 'Histoire (Moyen-Age)',
 'Histoire (Première Guerre mondiale)',
 'Histoire (Révolution française)',
 'Histoire (Seconde Guerre mondiale)',
 'Histoire (XIX° siècle)',
 'Histoire (XVIII° siècle)',
 'Histoire (XVII° siècle)',
 'Histoire (XVI° siècle)',
 'Histoire (XV° siècle)',
 'Histoire (XXI° siècle)',
 'Histoire (XX° siècle)',
 "Histoire de l'art",
 'Hiver',
 'Holocauste',
 'Homosexualité',
 'Hongrie',
 'Honte/Gêne',
 'Hôpital',
 'Humour',
 'Hygiène/Propreté',
 'Identité',
 'Illettrisme',
 'Illustration',
 'Imagier',
 'Imagination/Fabulation',
 'Immigration/Emigration',
 'Immortalité',
 'Imprimerie',
 'Inca/Aztèque',
 'Inde',
 'Indien',
 'Individualisme',
 'Indonésie',
 'Inégalité/Injustice',
 'Information/Actualité',
 'Insecte',
 'Instrument de musique',
 'Intégration',
 'Interculturalité',
 'Internet',
 'Inuit',
 'Irak',
 'Iran',
 'Irlande',
 'Islande',
 'Israël',
 'Italie',
 'Jalousie',
 'Jamaïque',
 'Japon',
 'Jardin',
 'Jardinage/Potager',
 'Jazz',
 "Jeu d'optique / Illusion d'optique",
 'Jeu de doigts',
 'Jeu de mots',
 'Jeu vidéo',
 'Jeu/Jouet',
 'Jeux Olympiques',
 'Journal intime',
 'Journalisme',
 'Judaïsme',
 'Jumeaux/Jumelles',
 'Justice',
 'Kamishibaï',
 'Kangourou',
 'Koala',
 'Kosovo',
 'La Réunion',
 "Lac/Cours d'eau",
 'Langue des signes',
 'Langue étrangère (apprentissage)',
 'Langue/Langage',
 'Lapin/Lièvre',
 "Lecture de l'image",
 'Légende arthurienne',
 'Légendes/Merveilleux',
 'Légume',
 'Lemming',
 'Léopard',
 'Lever/Réveil',
 'Liban',
 'Liberté',
 'Librairie',
 'Licorne',
 'Lion',
 'Liste',
 'Littérature',
 'Littérature de jeunesse',
 'Lituanie',
 'Livre animé',
 'Livre-jeu',
 'Livre/Lecture',
 'Londres',
 'Loup',
 'Loup-garou',
 'Lumière',
 'Lune',
 'Lutin',
 'Luxembourg',
 'Lycée',
 'Madagascar',
 'Magasin/Commerce',
 'Magie/Magicien',
 'Maladie',
 'Malaisie',
 'Mali',
 'Maltraitance',
 'Mammouth',
 'Manège',
 'Manga',
 'Manipulation',
 'Manipulation génétique/Clonage',
 'Mariage',
 'Mariage forcé',
 'Marionnette/Guignol',
 'Maroc',
 'Masque',
 'Mathématique',
 'Méchanceté/Cruauté',
 'Médecin/Docteur',
 'Médicament',
 'Mélancolie',
 'Mensonge/Vérité',
 'Menstruation/Règles',
 'Mer/Océan',
 'Métamorphose',
 'Météo/Climat',
 'Métier',
 'Métro',
 'Mexique',
 'Mille et une nuits',
 'Mine/Mineur',
 'Misère/Pauvreté',
 'Mode',
 'Monde arabe',
 'Mongolie',
 'Monstre',
 'Montagne',
 'Moquerie',
 'Mort/Deuil',
 'Mouche',
 'Mouette',
 'Moulin',
 'Mouton/Chèvre',
 'Moyen-Orient',
 'Musée',
 'Musique',
 'Mystère',
 'Mythologie',
 'Naissance/maternité',
 'Napoléon',
 'Narrateur/Narration',
 'Nature',
 'Naufrage',
 'Neige',
 'Népal',
 'New York',
 'Noël',
 'Normandie',
 'Norvège',
 'Nourrice/Baby-sitter',
 'Nourriture',
 'Nouvelle-Calédonie',
 'Nouvelle-Zélande',
 'Nouvelles technologies',
 'Nuit',
 'Obésité',
 'Océanie',
 'Odorat',
 'Oeuf',
 'Ogre/Géant',
 'Oiseau',
 'Opéra',
 'Ordinateur/Informatique',
 'Origami',
 'Orphelin/Orphelinat',
 'Ouïe',
 'Ours',
 'Paix/Pacifisme',
 'Pakistan',
 'Palestine',
 'Panda',
 'Papillon',
 'Pâques',
 'Paresse',
 'Parfum',
 'Paris',
 'Parodie',
 'Partage',
 'Paternité',
 'Patience/Impatience',
 'Pays-Bas',
 'Paysage',
 'Pêche/Pêcheur',
 'Pédagogie',
 'Pédophilie',
 'peintre',
 'Peinture',
 'Père Noël',
 'Pérou',
 'Perse',
 'Persévérance',
 'Petite enfance',
 'Peur',
 'Peur du noir/Cauchemar',
 'Philosophie',
 'Phoque/Morse',
 'Photographie',
 'Pingouin/Manchot',
 'Pique-nique',
 'Pirate/Corsaire',
 'Piscine',
 'Planète',
 'Plante',
 'Pluie/Orage',
 'Poésie',
 'Poison',
 'Poisson',
 'Policier',
 'Politesse/Bienséance',
 'Politique',
 'Pollution',
 'Pologne',
 'Polynésie',
 'Pompier',
 'Port',
 'Portrait',
 'Portugal',
 'Poule/Poussin/Coq',
 'Poupée',
 'Pouvoir',
 'Poux',
 'Pratiques artistiques et culturelles',
 'Préhistoire',
 'Préjugés',
 'Prénom/Surnom',
 'Prince/Princesse',
 'Printemps',
 'Prison',
 'Prix littéraire',
 'Promenade',
 'Provence',
 'Proverbe',
 'Psychologie/Psychanalyse',
 'Publicité',
 'Puce',
 'Punition/Fessée',
 'Québec',
 'Questions',
 'Racisme',
 'Racket',
 'Réfugié',
 'Relation Enfant/Adulte',
 'Relation Enfant/Animal',
 'Relation Frère/Soeur',
 'Relation Grand-Mère/Enfant',
 'Relation Grand-Parent/Enfant',
 'Relation Grand-Père/Enfant',
 'Relation Homme/Animal',
 'Relation Mère/Enfant',
 'Relation Mère/Fille',
 'Relation Mère/Fils',
 'Relation Père/Enfant',
 'Relation Père/Fille',
 'Relation Père/Fils',
 'Relation Personne âgée/Enfant',
 'Religion',
 'Renard',
 'Résistance',
 'Respect',
 'Responsabilité',
 'Rêve',
 'Révolte',
 'Richesse',
 'Ritournelle/Répétition',
 'Rivalité/Dispute',
 'Robot',
 'Roi/Reine',
 'Roman',
 'Roman historique',
 'Roman policier',
 'Roumanie',
 'Rue',
 'Rumeur',
 'Ruse',
 'Russie',
 'Rwanda',
 'Sagesse',
 'Saint-Nicolas',
 'Saison/Année/Cycle des saisons',
 'Santé',
 'Sapin',
 'Science',
 'Science-Fiction',
 'Sculpture',
 'Secret',
 'Secte',
 'Sécurité routière',
 'Ségrégation raciale',
 'Sénégal',
 'Séparation/Divorce',
 'Serpent',
 'Sexualité/Education sexuelle',
 'Sida',
 'Silence',
 'Singe',
 'Société',
 'Soleil',
 'Solitude',
 'Sommeil',
 'Son/Eveil sonore',
 'Sorcier/Sorcellerie',
 'Souffrance',
 'Souris/Rat/Mulot',
 'Souvenir/Mémoire',
 'Spéléologie',
 'Spiritualité',
 'Sport',
 'Steampunk',
 'Sucre',
 'Suède',
 'Suicide',
 'Suisse',
 'Super-héros',
 'Superstition',
 'Surdité',
 'Survie',
 'Suspense',
 'Syrie',
 'Tabac/Tabagisme',
 'Taïwan',
 'Taupe',
 'Téléphone',
 'Télévision/Radio',
 'Temps',
 'Terre',
 'Terrorisme',
 'Thaïlande',
 'Théâtre',
 "Théâtre d'ombres",
 'Tibet',
 'Tigre',
 'Timidité',
 'Tolérance',
 'Tortue',
 'Toucher/Matière',
 'Tradition',
 'Traduction',
 'Train/Gare',
 'Transidentité',
 'Transport/Moyens de transport',
 'Travail',
 'Travail des enfants',
 'Trésor',
 'Trisomie',
 'Tristesse/Chagrin',
 'Tsigane',
 'Tunisie',
 'Turquie',
 'Typographie',
 'Tyrannie',
 'Ukraine',
 'Univers/Système solaire',
 'Vacances',
 'Vache',
 'Vampire',
 'Végétarisme/Végétalisme',
 'Vénézuela',
 'Vengeance',
 'Venise',
 'Vent',
 'Vêtement',
 'Vie quotidienne',
 'Vieillesse',
 'Vietnam',
 'Viking',
 'Ville',
 'Viol',
 'Violence',
 'Voisinage',
 'Vol/Voleur',
 'Volcan',
 'Voyage',
 'Vue',
 'Yéti',
 'Yoga',
 'Zèbre',
 'Zombie',
 'Zoo',
 'Zoologie/Naturalisme']

In [14]:
ricochet_pretraite = [word.lower() for word in ricochet]

In [15]:
ricochet_pretraite

["abandon/sentiment d'abandon",
 'abécédaire/alphabet',
 'abeille',
 'aborigène',
 'absence',
 'absurde',
 'abus sexuel',
 'accident',
 'activités manuelles',
 "activités/cahier d'activités",
 "adaptation (d'œuvre littéraire)",
 'adolescence',
 'adoption',
 'adultère',
 'affection/tendresse',
 'affirmation de soi',
 'afghanistan',
 'afrique',
 'afrique du nord',
 'afrique du sud',
 'aide humanitaire',
 'alaska',
 'album',
 'album sans texte',
 'alcool/alcoolisme',
 'algérie',
 'alimentation/goût',
 'allemagne',
 'alphabétisation',
 'alpinisme',
 'amérique du nord',
 'amérique latine',
 'amitié',
 'amnésie',
 'amour',
 'analyse de texte',
 'ane',
 'ange',
 'animaux',
 'anniversaire/fête',
 'anorexie/boulimie',
 'antilles/caraïbes',
 'antiquité (gaule)',
 'antiquité (grèce)',
 'antiquité (rome)',
 'antisémitisme',
 'apartheid',
 'apprentissage',
 'apprentissage de la vie',
 'araignée',
 'arbre',
 'archéologie',
 'architecture',
 'argent/argent de poche',
 'argentine',
 'arménie',
 'art',

In [16]:
len(ricochet)

716

## échantillonnage aléatoire

In [139]:
# éléments dans la v1 de l'ensemble de données (composé de 30 ricochets, 30 écoles des loisirs)
# listes utilisées pour que la v2 de l'ensemble de données ne contiennent pas la v1
ricochet_v1_exclure = ['ours', 'monde arabe', 'naissance/maternité', 'révolte', 'londres', 'papillon', 'souvenir/mémoire', 'eau', 'sénégal', 'dragon', 'hygiène/propreté', "atelier d'écriture", 'alaska', 'poisson', 'astronomie', 'documentaire', 'débarquement', 'guatemala', 'sida', "lecture de l'image", 'pompier', 'nuit', 'univers/système solaire', 'peintre', 'souffrance', 'aide humanitaire', 'maltraitance', 'finlande', 'licorne', 'edition']
ecole_des_loisirs_v1_exclure = ['conte détourné', 'histoire : 18e siècle', 'aventure et mystère', 'théâtre contemporain', 'tchécoslovaquie', 'oman', 'genre', 'recherche de ses origines', 'préhistoire', 'père noël', 'rugby', 'balle, ballon', 'bolivie', 'roller', 'animaux domestiques', 'yémen', 'écrivain, écriture', 'biographie', 'hippocampe', 'arts martiaux', 'semaine', 'livre-jeu', 'histoire sans texte', 'cow-boy', 'transhumanisme', "conte détourné, boucle d'or et les 3 ours", 'relation enfant, adulte', 'conte détourné, le loup et les 7 cabris', 'regard des autres', 'iran']
v1_exclure = ricochet_v1_exclure + ecole_des_loisirs_v1_exclure

In [140]:
# Echantillon aléatoire de ricochet_raw en omettant les éléments de omit_list
ricochet_samples = random.sample([word for word in ricochet_pretraite if word not in v1_exclure], 20)

print("Echantillon aléatoire de ricochet_raw en omettant les éléments de omit_list")
print(len(ricochet_samples), ricochet_samples)

# Echantillon aléatoire de ecole_des_loisirs_raw en omettant les éléments de omit_list
ecole_des_loisirs_samples = random.sample([word for word in ecole_des_loisirs if word not in v1_exclure], 20)

print("Echantillon aléatoire de ecole_des_loisirs_raw en omettant les éléments de omit_list")
print(len(ecole_des_loisirs_samples), ecole_des_loisirs_samples)

Echantillon aléatoire de ricochet_raw en omettant les éléments de omit_list
20 ['champignon', 'famille recomposée', 'heure', 'tristesse/chagrin', 'sculpture', 'jeux olympiques', 'forme', 'odorat', 'mer/océan', 'différence', 'gourmandise', 'relation grand-père/enfant', 'drogue', 'aborigène', 'peinture', 'tibet', 'art brut', 'adoption', 'vent', 'pays-bas']
Echantillon aléatoire de ecole_des_loisirs_raw en omettant les éléments de omit_list
20 ['rivière, fleuve, étang', 'vétérinaire', 'Afghanistan', 'conte détourné, le petit poucet', 'fleur', 'jouets', 'naturaliste', 'hiver', 'épouvante', 'naissance, maternité', 'animaux divers', 'musée', 'physique, chimie', 'anniversaire et fête', 'champignons', 'pingouin, manchot', 'langage, jeux de mots', 'pionniers', 'jeux vidéo', 'Soudan']


## Assignation des évaluateurs
2 évaluateurs par échantillon pour 3 évaluateurs

In [19]:
def assigner_evaluateurs(echantillons):
    evaluateurs = ['Juliette', 'Michel', 'Karim']

    # Create a cycle iterator for the reviewers
    cycles_evaluateurs = itertools.cycle(evaluateurs)

    # Assign reviewers to each sample with a UUID
    assignations = {}
    for sample in echantillons:
        evaluateurs = [next(cycles_evaluateurs) for _ in range(2)] # Assign 2 reviewers per sample
        id_assignation = uuid.uuid4()
        assignations[id_assignation] = {'sample': sample, 'reviewers': evaluateurs}

    return assignations

# Example usage:
assignations_ricochet = assigner_evaluateurs(ricochet_samples)
assignations_ecole_des_loisirs = assigner_evaluateurs(ecole_des_loisirs_samples)


In [20]:
assignations_ricochet

{UUID('ab72df42-a1ac-4372-a22d-604f74c5e617'): {'sample': 'rue',
  'reviewers': ['Juliette', 'Michel']},
 UUID('0430a35d-22f5-462b-a200-9570bfa327ab'): {'sample': 'métro',
  'reviewers': ['Karim', 'Juliette']},
 UUID('a3224df5-09da-408e-99bd-6a79e1b6ed1e'): {'sample': 'biographie',
  'reviewers': ['Michel', 'Karim']},
 UUID('738cc5cc-d215-4368-b0d2-c825592f60a7'): {'sample': 'roumanie',
  'reviewers': ['Juliette', 'Michel']},
 UUID('b3e81a4f-fa05-4713-88bc-6180442c6db7'): {'sample': 'la réunion',
  'reviewers': ['Karim', 'Juliette']},
 UUID('52c3bf54-ed25-4aa1-b928-a1adc559694e'): {'sample': 'histoire (révolution française)',
  'reviewers': ['Michel', 'Karim']},
 UUID('893c67dd-d9b4-4596-9d96-ca2aacee7379'): {'sample': 'habitation/maison',
  'reviewers': ['Juliette', 'Michel']},
 UUID('93daa9de-b1d4-4fd0-bc5a-ac46738c7d1e'): {'sample': 'design',
  'reviewers': ['Karim', 'Juliette']},
 UUID('8ac8afd7-c62c-4980-98f6-878c79250577'): {'sample': 'danemark',
  'reviewers': ['Michel', 'Karim'

In [21]:
assignations_ecole_des_loisirs

{UUID('161938a1-0ef6-4133-924b-be6c2c0aadbc'): {'sample': 'Maghreb',
  'reviewers': ['Juliette', 'Michel']},
 UUID('ebe40b5f-c2e2-4f50-98df-0df175004e8c'): {'sample': 'imagier',
  'reviewers': ['Karim', 'Juliette']},
 UUID('aff248ea-0c8e-41b2-8c80-e5e3947194ae'): {'sample': 'manège',
  'reviewers': ['Michel', 'Karim']},
 UUID('b90052b6-5f18-43d8-a209-c3d2fc00d982'): {'sample': 'docteur, médecin',
  'reviewers': ['Juliette', 'Michel']},
 UUID('42618786-79f1-4df3-8fc3-29cf9eb1600c'): {'sample': 'photographie (livre de)',
  'reviewers': ['Karim', 'Juliette']},
 UUID('78d0a4da-4203-45c3-9f1c-404073dc5ea7'): {'sample': 'Mongolie',
  'reviewers': ['Michel', 'Karim']},
 UUID('327b50da-0713-4a8e-ba1a-f2e069f81e69'): {'sample': 'instinct, education',
  'reviewers': ['Juliette', 'Michel']},
 UUID('8ea89657-08c2-40c9-b66d-dd7e38dfa6b4'): {'sample': 'autobiographie',
  'reviewers': ['Karim', 'Juliette']},
 UUID('57cc835f-9b88-4836-9477-61bbc0d9e336'): {'sample': 'cycle des saisons',
  'reviewers':

## brassage aléatoire des assignations

In [22]:
assignations_brassees = list(assignations_ricochet.items()) + list(assignations_ecole_des_loisirs.items())
random.shuffle(assignations_brassees)

In [23]:
assignations_brassees

[(UUID('161938a1-0ef6-4133-924b-be6c2c0aadbc'),
  {'sample': 'Maghreb', 'reviewers': ['Juliette', 'Michel']}),
 (UUID('a3224df5-09da-408e-99bd-6a79e1b6ed1e'),
  {'sample': 'biographie', 'reviewers': ['Michel', 'Karim']}),
 (UUID('57cc835f-9b88-4836-9477-61bbc0d9e336'),
  {'sample': 'cycle des saisons', 'reviewers': ['Michel', 'Karim']}),
 (UUID('168fc314-04c3-465f-a743-c1ddb30b6179'),
  {'sample': 'premiers apprentissages', 'reviewers': ['Juliette', 'Michel']}),
 (UUID('06726c2a-6cdc-4752-95bf-7b49df3ac5c6'),
  {'sample': 'arbre', 'reviewers': ['Michel', 'Karim']}),
 (UUID('78d0a4da-4203-45c3-9f1c-404073dc5ea7'),
  {'sample': 'Mongolie', 'reviewers': ['Michel', 'Karim']}),
 (UUID('42618786-79f1-4df3-8fc3-29cf9eb1600c'),
  {'sample': 'photographie (livre de)', 'reviewers': ['Karim', 'Juliette']}),
 (UUID('0430a35d-22f5-462b-a200-9570bfa327ab'),
  {'sample': 'métro', 'reviewers': ['Karim', 'Juliette']}),
 (UUID('327b50da-0713-4a8e-ba1a-f2e069f81e69'),
  {'sample': 'instinct, education', 

## generation du fichier excel pour les evaluateurs

In [24]:
def ecrire_donnees_dans_excel(assignations, evaluateur):
    donnees_a_ecrire = []
    for id_assignations, info_assignation in assignations:
        sample = info_assignation['sample']
        evaluateurs = info_assignation['reviewers']
        if evaluateur in evaluateurs:
            donnees_a_ecrire.append({'UUID': id_assignations,
                        'Echantillon': sample, 
                        'Resultat 1': '', 
                        'Resultat 2': ''})
    
    df = pd.DataFrame(donnees_a_ecrire, columns=['UUID', 'Echantillon', 'Resultat 1', 'Resultat 2'])
    df.to_excel(f"{evaluateur}_assignations_v2.xlsx", index=False)   
    print("donneés ecrites")     


# decommenter pour ecrire
# ecrire_donnees_dans_excel(assignations_brassees, 'Juliette')
# ecrire_donnees_dans_excel(assignations_brassees, 'Michel')
# ecrire_donnees_dans_excel(assignations_brassees, 'Karim')


## excel qui réunit les assignations de tous les evaluateurs 

In [25]:
filename = 'assignations_generales_v2.xlsx' # fonction appelée 2 fois pour peupler le meme fichier
if os.path.exists(filename):
    os.remove(filename)

def peupler_donnees(donnees, assignations, source):
    for id_assignation, info_assignation in assignations.items():
        echantillon = info_assignation['sample']
        evaluateur = info_assignation['reviewers']
        donnees.append({'Uuid': id_assignation,
                     'Source': source, 
                     'Echantillon': echantillon, 
                     'Evaluateur 1': evaluateur[0], 
                     "Evaluateur 2": evaluateur[1], 
                     'Resultat 1': '', 
                     'Resultat 2': ''})
    return donnees
    
def ecrire_donnees_dans_excel_principal():
    # Define the data as a list of dictionaries

    donnees = []   
    donnees = peupler_donnees(donnees, assignations_ricochet, 'ricochet')
    donnees = peupler_donnees(donnees, assignations_ecole_des_loisirs, 'ecoles des loisirs')

    # Write the data to a CSV file
    data_frame = pd.DataFrame(donnees)
    data_frame.to_excel(filename, index=False)
    print("donneés ecrites")     

# decommenter pour ecrire
# ecrire_donnees_dans_excel_principal()


# lecture des jeux de données annotées sur excel

In [94]:
# Définir les options d'affichage pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', None)

In [141]:
# df = dataframe = objet pandas qui contient les données
df_michel = pd.read_excel('../book_themes_alignment_data/Michel_assignments_v1_v2.xlsx', keep_default_na=False, na_values=[''])
df_michel.rename(columns={'Echantillon': 'Echantillon (Michel)', 'Resultat 1': 'Resultat 1 (Michel)', 'Resultat 2': 'Resultat 2 (Michel)'}, inplace=True)
# print(df_michel)

In [142]:
df_karim = pd.read_excel('../book_themes_alignment_data/Karim_assignments_v1_v2.xlsx', keep_default_na=False, na_values=[''])
df_karim.rename(columns={'Echantillon': 'Echantillon (Karim)','Resultat 1': 'Resultat 1 (Karim)', 'Resultat 2': 'Resultat 2 (Karim)'}, inplace=True)
# print(df_karim)

In [143]:
df_juliette = pd.read_excel('../book_themes_alignment_data/Juliette_assignments_v1_v2.xlsx', keep_default_na=False, na_values=[''])
df_juliette.rename(columns={'Echantillon': 'Echantillon (Juliette)', 'Resultat 1': 'Resultat 1 (Juliette)', 'Resultat 2': 'Resultat 2 (Juliette)'}, inplace=True)
# print(df_juliette)

In [144]:
# Fusionner les dataframes
df_fusion = pd.concat([df_michel.set_index('UUID'), df_juliette.set_index('UUID'), df_karim.set_index('UUID')], axis=1)
echantillons_fusionnés = pd.concat([df_fusion["Echantillon (Juliette)"], df_fusion["Echantillon (Karim)"], df_fusion["Echantillon (Michel)"]]).dropna()
echantillons_fusionnés = echantillons_fusionnés.drop_duplicates()
df_fusion['Echantillon'] = echantillons_fusionnés
df_fusion.drop(columns=['Echantillon (Juliette)', 'Echantillon (Karim)', 'Echantillon (Michel)'], inplace=True)
df_fusion = df_fusion.dropna(subset=['Echantillon']) # cas utile: finlande dans v1 et v2 car présent dans Ricochet et Ecole des loisirs: Echantillon devient NaN et pas utile de le garder (car meme code Thema predit par evaluateurs)
print(df_fusion)

                                      Resultat 1 (Michel) Resultat 2 (Michel) Resultat 1 (Juliette) Resultat 2 (Juliette) Resultat 1 (Karim) Resultat 2 (Karim)                                Echantillon
UUID                                                                                                                                                                                                      
0f660312-e967-40fe-9a52-2cbed0ca23f8                   SF                 NaN                   NaN                   NaN                 SF                NaN                              balle, ballon
1885575b-e89a-4d0b-8474-9b98b049e634                  WNG                 NaN                   NaN                   NaN                WNG                NaN                        animaux domestiques
0d8703ec-db50-4936-9808-3d1e3befafe3                1HFDS                 NaN                   NaN                   NaN              1HFDS                NaN                             

In [145]:
df_michel_karim = df_fusion.loc[:, ['Echantillon', 'Resultat 1 (Michel)', 'Resultat 1 (Karim)']]
df_michel_karim = df_michel_karim.dropna()
print(df_michel_karim)

                                                  Echantillon Resultat 1 (Michel) Resultat 1 (Karim)
UUID                                                                                                
0f660312-e967-40fe-9a52-2cbed0ca23f8            balle, ballon                  SF                 SF
1885575b-e89a-4d0b-8474-9b98b049e634      animaux domestiques                 WNG                WNG
0d8703ec-db50-4936-9808-3d1e3befafe3                  sénégal               1HFDS              1HFDS
fbad8376-7a5b-4e0a-95d3-8edfda117779       atelier d'écriture                 CBW                CBW
3c436173-9bb3-47f5-80ab-405328bb397f                  edition               KNTP1              KNTP1
0703fbf6-f436-4cf6-913a-49295b511ccf             maltraitance                YXQD               YXQD
564404fe-7d19-447e-b4c7-b257188ee07b               biographie                 DNB                YNB
6e702f37-fd76-45a1-8ae1-3b9be5bb5bc4      aventure et mystère                YFCF          

In [146]:
df_michel_juliette = df_fusion.loc[:, ['Echantillon', 'Resultat 1 (Michel)', 'Resultat 1 (Juliette)']]
df_michel_juliette = df_michel_juliette.dropna()
print(df_michel_juliette)

                                                                  Echantillon Resultat 1 (Michel) Resultat 1 (Juliette)
UUID                                                                                                                   
a2492fac-993c-46c2-817b-67bf4b9fd16f                                livre-jeu               YBCS2                   YBG
f717074b-f95b-4ac7-a704-16ec89e8850b                                     ours                WNCF               YNNJV23
83d95dc7-2a1d-45e6-a6b0-efdabb296555                     théâtre contemporain                 DDC                   DDC
9dd6ba61-b47c-443a-8cab-7dc358fd1eeb                           transhumanisme                 UBJ                   FLP
be6e6fe5-9ea4-410a-ab44-b4e78e8339c3                                    genre                JBSF                   YXC
71f9881c-fbc5-4c5e-9c1d-983d93f35315                                   alaska        1KBB-US-WPNA          1KBB-US-WPNA
74977d39-4cd9-4abc-b91d-44150fe57676    

In [147]:
df_karim_juliette = df_fusion.loc[:, ['Echantillon', 'Resultat 1 (Karim)', 'Resultat 1 (Juliette)']]
df_karim_juliette = df_karim_juliette.dropna()
print(df_karim_juliette)

                                                                     Echantillon Resultat 1 (Karim) Resultat 1 (Juliette)
UUID                                                                                                                     
6122a480-817b-445a-b993-65e65accaa50                     univers/système solaire                PGM                   PGS
e9aabaf3-9be3-42cc-a210-e955048a7468                          écrivain, écriture               DNBL                   YNL
56a52e12-2040-4303-af7e-e921cd6dc7f5                       histoire : 18e siècle                YNH                    NH
1d4bf095-0667-4683-80e4-b9981e1b967e                                débarquement               JWCK                  JWCK
eb85b54c-76cd-4a63-84a2-dffa891bebd4                            aide humanitaire               JKSR                  JKSR
6383a51e-fcce-496d-9b4f-008a44c7570c                                       rugby              YNWD7                 YNWD7
efe3f8c2-80fe-4f90-ba11-

# kappa: comparaison des annotation entre les différents évaluateurs

In [148]:
# la lettre Y dans la 1ere position du code thema signifie que c'est un thème de jeunesse
def retirer_jeunesse_thema(list_a, list_b):
    assert(len(list_a) == len(list_b))
    for i in range (len(list_a)):
        if list_a[i][0] == 'Y':
            list_a[i] = list_a[i][1:]
        if list_b[i][0] == 'Y':
            list_b[i] = list_b[i][1:]
    

In [149]:
# ex: evaluateur 1 choisit YFCF et evaluateur 2 choisit YFC
# on remplace YFCF par YFC car YFC est le plus court et donc le plus général
def list_theme_abstraction(list_a, list_b):
    assert(len(list_a) == len(list_b))
    for i in range(len(list_a)):
        if list_a[i] in list_b[i]:
            list_a[i] = list_b[i]
        if list_b[i] in list_a[i]:
            list_b[i] = list_a[i]

In [150]:
michel = df_michel_karim["Resultat 1 (Michel)"].tolist()
karim = df_michel_karim["Resultat 1 (Karim)"].tolist()
print(len(michel))
print(len(karim))
kappa = sklearn.metrics.cohen_kappa_score(michel, karim)
print("Michel & Karim", kappa, len(michel))

31
31
Michel & Karim 0.4756871035940803 31


In [151]:
retirer_jeunesse_thema(michel, karim)
list_theme_abstraction(michel, karim)
kappa = sklearn.metrics.cohen_kappa_score(michel, karim)
print("Michel & Karim avec abstraction theme", kappa, len(michel))

Michel & Karim avec abstraction theme 0.5726405090137857 31


In [152]:
michel = df_michel_juliette["Resultat 1 (Michel)"].tolist()
juliette = df_michel_juliette["Resultat 1 (Juliette)"].tolist()

kappa = sklearn.metrics.cohen_kappa_score(michel, juliette)
print("Michel & Juliette", kappa, len(michel))

Michel & Juliette 0.5211267605633803 34


In [153]:
retirer_jeunesse_thema(michel, juliette)
list_theme_abstraction(michel, juliette)
kappa = sklearn.metrics.cohen_kappa_score(michel, juliette)
print("Michel & Juliette avec abstraction theme", kappa, len(michel))

Michel & Juliette avec abstraction theme 0.5211267605633803 34


In [154]:
karim = df_karim_juliette["Resultat 1 (Karim)"].tolist()
juliette = df_karim_juliette["Resultat 1 (Juliette)"].tolist()

kappa = sklearn.metrics.cohen_kappa_score(karim, juliette)
print("Juliette & Karim", kappa, len(karim))

Juliette & Karim 0.43333333333333335 34


In [155]:
retirer_jeunesse_thema(karim, juliette)
list_theme_abstraction(karim, juliette)
kappa = sklearn.metrics.cohen_kappa_score(karim, juliette)
print("Michel & Juliette avec abstraction theme", kappa, len(karim))

Michel & Juliette avec abstraction theme 0.4626865671641791 34


# creation d'un dictionnaire pour les données de références (DR)
[clé=nom_theme, valeur=list(code_theme)]

In [25]:
theme_dr = {} # dr = données référence
for index, row in df_fusion.iterrows():
    resultats_ligne_df = row.filter(like='Resultat').dropna()
    if not isinstance(row["Echantillon"], str):
        print(row)
        continue
    theme_dr[row["Echantillon"]] = resultats_ligne_df.tolist()
theme_dr

NameError: name 'df_fusion' is not defined

In [26]:
chemin_fichier = '../book_themes_alignment_data/themes_jeu_validation_donnees_references.json'

# Save the JSON string to a file
with open(chemin_fichier, 'w') as file:
    json.dump(theme_dr, file)

# Print the file path
print(f"Données références sauvegardées à: {chemin_fichier}")


Données références sauvegardées à: ../book_themes_alignment_data/themes_jeu_validation_donnees_references.json


# alignement thesaurus

## import données références

In [3]:
chemin_fichier = '../book_themes_alignment_data/themes_jeu_validation_donnees_references.json'

with open(chemin_fichier, "r") as file:
    themes_jeu_validation_dr = json.load(file) # dr = données references

themes_jeu_validation_dr

{'balle, ballon': ['SF', 'SF'],
 'animaux domestiques': ['WNG', 'WNG'],
 'sénégal': ['1HFDS', '1HFDS'],
 'livre-jeu': ['YBCS2', 'YBG'],
 "atelier d'écriture": ['CBW', 'CBW'],
 'edition': ['KNTP1', 'KNTP1'],
 'ours': ['WNCF', 'YNNJV23'],
 'théâtre contemporain': ['DDC', 'DDC'],
 'maltraitance': ['YXQD', 'YXQD'],
 'transhumanisme': ['UBJ', 'FLP', 'UBJ'],
 'genre': ['JBSF', 'YXC'],
 'alaska': ['1KBB-US-WPNA', '1KBB-US-WPNA'],
 'biographie': ['DNB', 'YNB'],
 'bolivie': ['1KLSL', '1KLSL'],
 'conte détourné, le loup et les 7 cabris': ['YFHG', 'YFHG'],
 'aventure et mystère': ['YFCF', 'YFC', 'YNX'],
 'conte détourné': ['YFHG', 'YFHG'],
 'yémen': ['1FBXY', '1FBXY'],
 'hippocampe': ['WNCS1', 'N/A'],
 'cow-boy': ['FJW', 'FRJ'],
 'dragon': ['VXQM1', 'YNXB1'],
 'père noël': ['5HPD', 'N/A'],
 'sida': ['MJCJ2', 'MJCJ2'],
 'souvenir/mémoire': ['JMR', 'VSPT'],
 'pompier': ['JKSW2', 'JKSW2'],
 'iran': ['1FBN', '1FBN'],
 'semaine': ['YBLJ', '5HCK'],
 'finlande': ['1DNF', '1DNF'],
 'peintre': ['AGB', 'YN

## import des noeuds thema

## prétraitement : prendre le premier mot, le réduire et supprimer les caractères spéciaux

In [5]:
with open("../book_themes_alignment_data/thema_code_dict.json", "r") as file:
    thema_code_dict = json.load(file)

thema_code_dict

{'A': {'CodeValue': 'A',
  'CodeDescription': 'Arts',
  'CodeNotes': 'Utilisez tous les codes A* pour les ouvrages spécialisés et généraux, qu’ils soient richement illustrés ou majoritairement textuel. Préférez les codes WF* pour les ouvrages liés à un loisir ou un passe-temps, en le complétant au besoin par le(s) code(s) A*. Utilisez au besoin tous les codes A* avec d’autres codes et qualificateurs, particulièrement avec les qualificateurs de style 6*, de lieux 1* et historiques 3*',
  'CodeParent': '',
  'IssueNumber': 1,
  'Modified': 1.4},
 'AB': {'CodeValue': 'AB',
  'CodeDescription': 'Arts : généralités',
  'CodeNotes': '',
  'CodeParent': 'A',
  'IssueNumber': 1,
  'Modified': 1.5},
 'ABA': {'CodeValue': 'ABA',
  'CodeDescription': 'Théorie de l’art',
  'CodeNotes': 'Lié : QDTN',
  'CodeParent': 'AB',
  'IssueNumber': 1,
  'Modified': ''},
 'ABC': {'CodeValue': 'ABC',
  'CodeDescription': 'Conservation, restauration et entretien d’œuvres d’art',
  'CodeNotes': 'Utilisez tous le

In [5]:
noeuds_thema_pretraites = {}
for code, theme in thema_code_dict.items():
    # Preprocess the code description
    noeuds_thema_pretraites[code] = copy.deepcopy(theme)
    noeuds_thema_pretraites[code]["CodeDescription"] = theme["CodeDescription"].lower()
    noeuds_thema_pretraites[code]["CodeNotes"] = theme["CodeNotes"].lower()

noeuds_thema_pretraites

{'A': {'CodeValue': 'A',
  'CodeDescription': 'arts',
  'CodeNotes': 'utilisez tous les codes a* pour les ouvrages spécialisés et généraux, qu’ils soient richement illustrés ou majoritairement textuel. préférez les codes wf* pour les ouvrages liés à un loisir ou un passe-temps, en le complétant au besoin par le(s) code(s) a*. utilisez au besoin tous les codes a* avec d’autres codes et qualificateurs, particulièrement avec les qualificateurs de style 6*, de lieux 1* et historiques 3*',
  'CodeParent': '',
  'IssueNumber': 1,
  'Modified': 1.4},
 'AB': {'CodeValue': 'AB',
  'CodeDescription': 'arts : généralités',
  'CodeNotes': '',
  'CodeParent': 'A',
  'IssueNumber': 1,
  'Modified': 1.5},
 'ABA': {'CodeValue': 'ABA',
  'CodeDescription': 'théorie de l’art',
  'CodeNotes': 'lié : qdtn',
  'CodeParent': 'AB',
  'IssueNumber': 1,
  'Modified': ''},
 'ABC': {'CodeValue': 'ABC',
  'CodeDescription': 'conservation, restauration et entretien d’œuvres d’art',
  'CodeNotes': 'utilisez tous le

## alignement thesaurus naif: correspondance exacte

In [6]:
def predire_correspondance_exacte(thema_dict):
    predictions_correspondance_exacte = {}
    compte_correspondances = 0
    correspondances = []
    jeu_donnees_complet = []

    for theme_a_evaluer in themes_jeu_validation_dr.keys():
        jeu_donnees_complet.append(theme_a_evaluer)
        for thema_code, thema_valeur in thema_dict.items():
            thema_valeur = thema_valeur["CodeDescription"]
            if theme_a_evaluer == thema_valeur and theme_a_evaluer not in predictions_correspondance_exacte.values():
                correspondances.append(theme_a_evaluer)
                print(theme_a_evaluer, "->", thema_valeur)
                compte_correspondances += 1
                predictions_correspondance_exacte[theme_a_evaluer] = thema_code
    
    sans_correspondances = list(set(jeu_donnees_complet) - set(correspondances))
    print("jeu données complet", len(jeu_donnees_complet))
    print(jeu_donnees_complet)
    print("correspgondances", len(correspondances))
    print(correspondances)
    print("sans correspondance", len(sans_correspondances))
    print(sans_correspondances)

    return predictions_correspondance_exacte
predictions_correspondance_exacte = predire_correspondance_exacte(noeuds_thema_pretraites)

sénégal -> sénégal
alaska -> alaska
bolivie -> bolivie
yémen -> yémen
iran -> iran
finlande -> finlande
guatemala -> guatemala
préhistoire -> préhistoire
oman -> oman
corse -> corse
ouzbékistan -> ouzbékistan
irak -> irak
pologne -> pologne
arts martiaux -> arts martiaux
tchécoslovaquie -> tchécoslovaquie
madagascar -> madagascar
ecosse -> ecosse
jeu données complet 100
['balle, ballon', 'animaux domestiques', 'sénégal', 'livre-jeu', "atelier d'écriture", 'edition', 'ours', 'théâtre contemporain', 'maltraitance', 'transhumanisme', 'genre', 'alaska', 'biographie', 'bolivie', 'conte détourné, le loup et les 7 cabris', 'aventure et mystère', 'conte détourné', 'yémen', 'hippocampe', 'cow-boy', 'dragon', 'père noël', 'sida', 'souvenir/mémoire', 'pompier', 'iran', 'semaine', 'finlande', 'peintre', 'relation enfant, adulte', 'papillon', 'souffrance', 'nuit', 'guatemala', 'préhistoire', 'astronomie', 'révolte', 'naissance/maternité', 'documentaire', 'oman', 'clown', 'corse', 'castor', 'cosmona

In [40]:
import sys
from pprint import pprint
epsilon = sys.float_info.epsilon


def calculer_metriques(predictions_themes, donnees_references=themes_jeu_validation_dr):
    print("len(theme evaluation)", len(predictions_themes))  # contient les evaluations positives du modele
    print("len(donnees_references)", len(donnees_references)) # contient les données de références de tout le jeu de données de validation

    VP = epsilon # eviter division par 0
    VP_liste = []
    FP = 0
    FP_liste = []
    for prediction_theme_nom, prediction_theme_code in predictions_themes.items():
        if prediction_theme_nom in donnees_references and prediction_theme_code in donnees_references[prediction_theme_nom]:
            VP_liste.append((prediction_theme_nom, prediction_theme_code, donnees_references[prediction_theme_nom]))
            VP += 1
        elif prediction_theme_nom in donnees_references and prediction_theme_code not in donnees_references[prediction_theme_nom]:
            prediction_theme_nom_thema = thema_code_dict[prediction_theme_code]["CodeDescription"] # nom du theme evalué par le modele dans théma
            jeu_validation_theme_code = donnees_references[prediction_theme_nom][0]
            if jeu_validation_theme_code != 'N/A' :
                jeu_validation_nom_theme = thema_code_dict[jeu_validation_theme_code]["CodeDescription"] # nom du theme des données de rérérence dans thema
                FP_liste.append(f"{prediction_theme_nom} -> {prediction_theme_nom_thema} vs φ={jeu_validation_nom_theme} | {prediction_theme_code} vs φ={jeu_validation_theme_code}")
            else:
                FP_liste.append(f"{prediction_theme_nom} -> {prediction_theme_nom_thema} vs φ=N/A | {prediction_theme_code} vs φ={jeu_validation_theme_code}")


            FP += 1

    FN = len(donnees_references) - len(predictions_themes)  # FN = N =  tout le jeu de donneés - P
    print("VP", int(VP))
    pprint(VP_liste)
    print("FP", FP)
    pprint(FP_liste)
    # les predictions du modele correspondent aux predictions positives: nous présumons que tous les mots-clés ont un thème correspondant dans théma      
    assert(FP == len(predictions_themes) - int(VP)) # FP = P - VP
    assert(len(donnees_references) == VP + FN + FP)


    accuracy = (VP / len(donnees_references))
    precision = VP / (VP + FP)
    recall = VP / (VP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)

    print("VP", int(VP))
    print("FP", FP)
    print("FN", FN)

    print("accuracy", accuracy)
    print("precision", precision)
    print("rappel", recall)
    print("f1_score", f1_score)
    return VP_liste, FP_liste

In [33]:
calculer_metriques(predictions_correspondance_exacte)

len(theme evaluation) 17
len(donnees_references) 100
VP 16
[('sénégal', '1HFDS', ['1HFDS', '1HFDS']),
 ('alaska', '1KBB-US-WPNA', ['1KBB-US-WPNA', '1KBB-US-WPNA']),
 ('bolivie', '1KLSL', ['1KLSL', '1KLSL']),
 ('yémen', '1FBXY', ['1FBXY', '1FBXY']),
 ('iran', '1FBN', ['1FBN', '1FBN']),
 ('finlande', '1DNF', ['1DNF', '1DNF']),
 ('guatemala', '1KLCG', ['1KLCG', '1KLCG']),
 ('préhistoire', '3B', ['3B', '3B']),
 ('oman', '1FBXM', ['1FBXM', '1FBXM']),
 ('corse', '2ADH', ['1DDF-FR-I', '2ADH']),
 ('ouzbékistan', '1FCU', ['1FCU', '1FCU']),
 ('irak', '1FBQ', ['1FBQ', '1FBQ']),
 ('pologne', '1DTP', ['1DTP', '1DTP']),
 ('tchécoslovaquie', '1QBDK', ['1QBDK', '1QBDK']),
 ('madagascar', '1HSM', ['1HSM', '1HSM']),
 ('ecosse', '1DDU-GB-S', ['1DDU-GB-S', '1DDU-GB-S'])]
FP 1
['arts martiaux -> Arts martiaux vs φ=Documentaires jeunesse : arts martiaux | '
 'SRM vs φ=YNWJ']
VP 16
FP 1
FN 83
accuracy 0.16
precision 0.9411764705882353
rappel 0.16161616161616163
f1_score 0.2758620689655173


## levenshtein

In [9]:
import Levenshtein
def predire_levenshtein(thema_dict):
    predictions_levenshtein = {} # clé = nom theme, valeur = code theme
    nombre_predictions = 0
    tous_les_themes = []

    for theme_a_predire in themes_jeu_validation_dr.keys():
        # UPPER_DISTANCE_LIMIT = max(len(theme_to_evaluate) / 3, 0)
        LIMITE_SUPERIEURE_DISTANCE = 1
        tous_les_themes.append(theme_a_predire)
        min_distance = 100
        best_match = None
        for thema_code, thema_value in thema_dict.items():
            thema_value = thema_value["CodeDescription"]
            distance = Levenshtein.distance(theme_a_predire, thema_value)
            if distance < min_distance and distance <= LIMITE_SUPERIEURE_DISTANCE:
                min_distance = distance
                best_match = thema_code
        if min_distance <= LIMITE_SUPERIEURE_DISTANCE and best_match is not None and theme_a_predire not in predictions_levenshtein.values():
            print(min_distance, theme_a_predire, "->", thema_dict[best_match]["CodeDescription"])
            nombre_predictions += 1
            predictions_levenshtein[theme_a_predire] = best_match
    
    non_matches = list(set(tous_les_themes) - set(predictions_levenshtein.keys()))
    print("tous les themes", len(tous_les_themes))
    print(tous_les_themes)
    print("themes prédits", len(predictions_levenshtein.keys()))
    print(predictions_levenshtein.keys())
    print("themes non predits", len(non_matches))
    print(non_matches)

    return predictions_levenshtein

predictions_levenshtein = predire_levenshtein(noeuds_thema_pretraites)


0 sénégal -> sénégal
1 ours -> tours
0 alaska -> alaska
0 bolivie -> bolivie
0 yémen -> yémen
0 iran -> iran
0 finlande -> finlande
0 guatemala -> guatemala
0 préhistoire -> préhistoire
1 documentaire -> documentaires
0 oman -> oman
0 corse -> corse
0 ouzbékistan -> ouzbékistan
1 europe de l'est -> europe de l’est
0 irak -> irak
0 pologne -> pologne
0 arts martiaux -> arts martiaux
0 tchécoslovaquie -> tchécoslovaquie
1 eau -> pau
0 madagascar -> madagascar
0 ecosse -> ecosse
tous les themes 100
['balle, ballon', 'animaux domestiques', 'sénégal', 'livre-jeu', "atelier d'écriture", 'edition', 'ours', 'théâtre contemporain', 'maltraitance', 'transhumanisme', 'genre', 'alaska', 'biographie', 'bolivie', 'conte détourné, le loup et les 7 cabris', 'aventure et mystère', 'conte détourné', 'yémen', 'hippocampe', 'cow-boy', 'dragon', 'père noël', 'sida', 'souvenir/mémoire', 'pompier', 'iran', 'semaine', 'finlande', 'peintre', 'relation enfant, adulte', 'papillon', 'souffrance', 'nuit', 'guatema

In [34]:
calculer_metriques(predictions_levenshtein)

len(theme evaluation) 21
len(donnees_references) 100
VP 18
[('sénégal', '1HFDS', ['1HFDS', '1HFDS']),
 ('alaska', '1KBB-US-WPNA', ['1KBB-US-WPNA', '1KBB-US-WPNA']),
 ('bolivie', '1KLSL', ['1KLSL', '1KLSL']),
 ('yémen', '1FBXY', ['1FBXY', '1FBXY']),
 ('iran', '1FBN', ['1FBN', '1FBN']),
 ('finlande', '1DNF', ['1DNF', '1DNF']),
 ('guatemala', '1KLCG', ['1KLCG', '1KLCG']),
 ('préhistoire', '3B', ['3B', '3B']),
 ('documentaire', 'ATFR', ['YN', 'ATFR']),
 ('oman', '1FBXM', ['1FBXM', '1FBXM']),
 ('corse', '2ADH', ['1DDF-FR-I', '2ADH']),
 ('ouzbékistan', '1FCU', ['1FCU', '1FCU']),
 ("europe de l'est", '1DT', ['1DT', '1DT']),
 ('irak', '1FBQ', ['1FBQ', '1FBQ']),
 ('pologne', '1DTP', ['1DTP', '1DTP']),
 ('tchécoslovaquie', '1QBDK', ['1QBDK', '1QBDK']),
 ('madagascar', '1HSM', ['1HSM', '1HSM']),
 ('ecosse', '1DDU-GB-S', ['1DDU-GB-S', '1DDU-GB-S'])]
FP 3
['ours -> Tours vs φ=Vie sauvage : mammifères : généralités | 1DDF-FR-GDA vs '
 'φ=WNCF',
 'arts martiaux -> Arts martiaux vs φ=Documentaires jeu

## wiki2vec

In [11]:
from wikipedia2vec import Wikipedia2Vec
# Telecharger modele prentrainé sur https://wikipedia2vec.github.io/wikipedia2vec/pretrained/
# Charger le modèle pré-entraîné
wiki2vec = Wikipedia2Vec.load('../book_themes_alignment_data/frwiki_20180420_300d.pkl')

# Trouver le mot le plus similaire à un mot donné
mots_similaire = wiki2vec.most_similar(wiki2vec.get_word('chien'), 10)

# Print the similar words
for mot, similarite in mots_similaire:
    print(f"{mot}: {similarite}")

c:\Users\dre\Documents\projet biblio scolaire\Books-Python\books-python-env\Lib\site-packages\joblib\numpy_pickle.py:230: UserWarning: The memmapped array [[-0.3918146  -0.24315855 -0.29823107 ... -0.15211841  0.34864503
  -0.28443316]
 [-0.31431052 -0.05210092 -0.23165612 ...  0.33593094  0.21336684
  -0.2881443 ]
 [ 0.23390982 -0.00242238 -0.24155504 ...  0.10083728 -0.06711676
  -0.03356545]
 ...
 [-0.37162766 -0.5298218  -0.05777219 ... -0.2812191   0.15711616
   0.2854844 ]
 [-0.75576866 -0.14604965 -0.24931757 ... -0.32277733 -0.1752458
   0.3595142 ]
 [-0.4601417  -0.19192158 -0.281166   ... -0.36144966 -0.00496797
   0.08007599]] loaded from the file ../book_themes_alignment_data/frwiki_20180420_300d.pkl is not byte aligned. This may cause segmentation faults if this memmapped array is used in some libraries like BLAS or PyTorch. To get rid of this warning, regenerate your pickle file with joblib >= 1.2.0. See https://github.com/joblib/joblib/issues/563 for more details
  warni

<Word chien>: 1.0000001192092896
<Word chiens>: 0.7172577381134033
<Word chiot>: 0.6617892980575562
<Word chat>: 0.658769428730011
<Word pudelpointer>: 0.6494312286376953
<Word chienne>: 0.6439096927642822
<Entity Chien de chasse>: 0.6328124403953552
<Word caniche>: 0.6294683814048767
<Word drahthaar>: 0.6275166869163513
<Word snuppy>: 0.6268122792243958


In [12]:
def predire_wiki2vec(thema_dict, wiki2vec):

    predictions = {}
    SIMILARITE_COSINE_MINIMALE = -1

    for theme_a_predire in themes_jeu_validation_dr.keys():
        similarite_max = SIMILARITE_COSINE_MINIMALE
        meilleure_prediction = None
        for thema_code, thema_nom in thema_dict.items():
            thema_nom = thema_nom["CodeDescription"] # nom du theme dans thema
            try:
                similarity = utils.cosine_similarity(wiki2vec.get_word_vector(theme_a_predire), 
                                                     wiki2vec.get_word_vector(thema_nom))
            except KeyError:
                similarity = SIMILARITE_COSINE_MINIMALE
            if similarity > similarite_max:
                similarite_max = similarity
                meilleure_prediction = thema_code
        if meilleure_prediction is not None:
            print(theme_a_predire, "->", thema_dict[meilleure_prediction]["CodeDescription"], similarite_max)
            predictions[theme_a_predire] = meilleure_prediction
        
    return predictions

predictions_wiki2vec = predire_wiki2vec(noeuds_thema_pretraites, wiki2vec)

sénégal -> sénégal 0.99999994
edition -> evolution 0.44893438
ours -> yupik 0.384791
maltraitance -> négligence 0.56435037
transhumanisme -> bioéthique 0.5027729
genre -> protozoaire 0.38188612
alaska -> alaska 1.0000001
biographie -> histoire 0.46998644
bolivie -> bolivie 1.0
yémen -> yémen 1.0
hippocampe -> dada 0.3293102
dragon -> fléchette 0.37385482
sida -> vaccination 0.5347764
pompier -> camping 0.3426182
iran -> iran 1.0000001
semaine -> pâques 0.40029237
finlande -> finlande 1.0000001
peintre -> cloisonnisme 0.5071018
papillon -> natation 0.3748513
souffrance -> négligence 0.48267052
nuit -> lune 0.42426005
guatemala -> guatemala 0.99999994
préhistoire -> préhistoire 1.0000001
astronomie -> astrophysique 0.6789408
révolte -> mutinerie 0.65357226
documentaire -> documentaires 0.6659119
oman -> oman 1.0000001
clown -> halloween 0.38882792
corse -> corse 1.0
castor -> mercure 0.42985138
harcèlement -> négligence 0.5109072
hospitalisation -> hémodialyse 0.5978879
ouzbékistan -> ou

In [35]:
calculer_metriques(predictions_wiki2vec)

len(theme evaluation) 65
len(donnees_references) 100
VP 17
[('sénégal', '1HFDS', ['1HFDS', '1HFDS']),
 ('alaska', '1KBB-US-WPNA', ['1KBB-US-WPNA', '1KBB-US-WPNA']),
 ('bolivie', '1KLSL', ['1KLSL', '1KLSL']),
 ('yémen', '1FBXY', ['1FBXY', '1FBXY']),
 ('iran', '1FBN', ['1FBN', '1FBN']),
 ('finlande', '1DNF', ['1DNF', '1DNF']),
 ('guatemala', '1KLCG', ['1KLCG', '1KLCG']),
 ('préhistoire', '3B', ['3B', '3B']),
 ('documentaire', 'ATFR', ['YN', 'ATFR']),
 ('oman', '1FBXM', ['1FBXM', '1FBXM']),
 ('corse', '2ADH', ['1DDF-FR-I', '2ADH']),
 ('ouzbékistan', '1FCU', ['1FCU', '1FCU']),
 ('irak', '1FBQ', ['1FBQ', '1FBQ']),
 ('pologne', '1DTP', ['1DTP', '1DTP']),
 ('tchécoslovaquie', '1QBDK', ['1QBDK', '1QBDK']),
 ('madagascar', '1HSM', ['1HSM', '1HSM']),
 ('ecosse', '1DDU-GB-S', ['1DDU-GB-S', '1DDU-GB-S'])]
FP 48
['edition -> Evolution vs φ=Edition et commerce du livre | PSAJ vs φ=KNTP1',
 'ours -> Yupik vs φ=Vie sauvage : mammifères : généralités | 2JNBN vs φ=WNCF',
 'maltraitance -> Négligence vs 

## doc2vec

In [3]:
import spacy
modele_nlp_spacy = spacy.load('fr_core_news_lg') # modele

In [4]:
noeuds_thema_cache = {}
for code, theme in thema_code_dict.items():
    theme_temp = {}
    theme_temp["CodeDescription"] = theme["CodeDescription"].lower()
    theme_temp["CodeDescription_Latent"] =  modele_nlp_spacy(theme["CodeDescription"].lower())
    noeuds_thema_cache[code] = theme_temp

noeuds_thema_cache

NameError: name 'thema_code_dict' is not defined

In [16]:
def alignement_doc2vec(noeuds_thema, spacy_nlp, jeu_validation_dr):

    predictions = {}
    SIMILARITE_COSINE_MINIMALE = -1

    for theme_a_predire in jeu_validation_dr.keys():
        similarite_max = SIMILARITE_COSINE_MINIMALE
        meilleure_prediction = None
        theme_a_predire_latent = spacy_nlp(theme_a_predire)
        for thema_code, thema_valeur in noeuds_thema.items():
            thema_valeur_modele = thema_valeur["CodeDescription_Latent"] # nom theme thema dans l'espace latent du modele  
            similarite = theme_a_predire_latent.similarity(thema_valeur_modele)
            if similarite > similarite_max:
                similarite_max = similarite
                meilleure_prediction = thema_code
        if meilleure_prediction is not None:
            print(theme_a_predire, "->", noeuds_thema[meilleure_prediction]["CodeDescription"], similarite_max)
            predictions[theme_a_predire] = meilleure_prediction
        
    return predictions


predictions_doc2vec = alignement_doc2vec(noeuds_thema_cache, modele_nlp_spacy, themes_jeu_validation_dr) 

C:\Users\dre\AppData\Local\Temp\ipykernel_19260\462604653.py:12: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarite = theme_a_predire_latent.similarity(thema_valeur_modele)


balle, ballon -> fléchette 0.5571151654286275
animaux domestiques -> plantes sauvages et cueillette comestibles 0.7600863321665284
sénégal -> sénégal 1.0
livre-jeu -> techno-thriller 0.6226885881212054
atelier d'écriture -> logiciel d’illustration et de dessin 0.7637448133138999
edition -> evolution 0.6820361120421728
ours -> documentaires jeunesse : ours 0.5186113587089525
théâtre contemporain -> théâtre 0.9187365837867748
maltraitance -> violence domestique 0.7488391574613409
transhumanisme -> constructivisme 0.8150649623147851
genre -> fantasy humoristique 0.532994392672803
alaska -> alaska 1.0
biographie -> historiographie 0.7384783147127199
bolivie -> bolivie 1.0
conte détourné, le loup et les 7 cabris -> le tibre et ses affluents 0.7470662811876552
aventure et mystère -> cosmologie et univers 0.8536465099852968
conte détourné -> roman sentimental historique 0.6771777351524145
yémen -> yémen 1.0
hippocampe -> corte 0.4393069802783025
cow-boy -> saxe-anhalt 0.675274198619341
dragon

In [36]:
calculer_metriques(predictions_doc2vec)

len(theme evaluation) 100
len(donnees_references) 100
VP 22
[('sénégal', '1HFDS', ['1HFDS', '1HFDS']),
 ('alaska', '1KBB-US-WPNA', ['1KBB-US-WPNA', '1KBB-US-WPNA']),
 ('bolivie', '1KLSL', ['1KLSL', '1KLSL']),
 ('yémen', '1FBXY', ['1FBXY', '1FBXY']),
 ('père noël', '5HPD', ['5HPD', 'N/A']),
 ('sida', 'MJCJ2', ['MJCJ2', 'MJCJ2']),
 ('iran', '1FBN', ['1FBN', '1FBN']),
 ('finlande', '1DNF', ['1DNF', '1DNF']),
 ('guatemala', '1KLCG', ['1KLCG', '1KLCG']),
 ('préhistoire', '3B', ['3B', '3B']),
 ('documentaire', 'ATFR', ['YN', 'ATFR']),
 ('oman', '1FBXM', ['1FBXM', '1FBXM']),
 ('corse', '2ADH', ['1DDF-FR-I', '2ADH']),
 ('ouzbékistan', '1FCU', ['1FCU', '1FCU']),
 ("europe de l'est", '1DT', ['1DT', '1DT']),
 ('irak', '1FBQ', ['1FBQ', '1FBQ']),
 ('pologne', '1DTP', ['1DTP', '1DTP']),
 ('aide humanitaire', 'JKSR', ['JKSR', 'JKSR']),
 ('tchécoslovaquie', '1QBDK', ['1QBDK', '1QBDK']),
 ('londres', '1DDU-GB-ESL', ['JPT', '1DDU-GB-ESLF', '1DDU-GB-ESL']),
 ('madagascar', '1HSM', ['1HSM', '1HSM']),
 ('e

In [18]:
def alignement_doc2vec_top_n(noeuds_thema, spacy_nlp, jeu_validation_dr, top_n=5):
    predictions = {}

    for theme_a_predire in jeu_validation_dr.keys():
        similarites = []
        theme_a_predire_latent = spacy_nlp(theme_a_predire)
        for thema_code, thema_valeur in noeuds_thema.items():
            thema_valeur_modele = thema_valeur["CodeDescription_Latent"]
            similarite = theme_a_predire_latent.similarity(thema_valeur_modele)
            similarites.append((thema_code, thema_valeur["CodeDescription"], similarite))
        
        similarites.sort(key=lambda x: x[2], reverse=True)
        top3_similarites = similarites[:top_n]
        predictions[theme_a_predire] = top3_similarites
    
    return predictions


predictions_doc2vec_top5 = alignement_doc2vec_top_n(noeuds_thema_cache, modele_nlp_spacy, themes_jeu_validation_dr)

C:\Users\dre\AppData\Local\Temp\ipykernel_19260\1975866870.py:9: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarite = theme_a_predire_latent.similarity(thema_valeur_modele)


In [19]:
pprint(predictions_doc2vec_top5)

{'aide humanitaire': [('JKSR', 'aide et programmes d’aide', 0.7584345119582455),
                      ('VFB', 'sécurité personnelle', 0.6368474982181922),
                      ('JNSR',
                       'education et enseignement correctif',
                       0.6331218425714127),
                      ('JKS',
                       'assistance sociale et services sociaux',
                       0.628791021054243),
                      ('MBPM',
                       'administration médicale et gestion',
                       0.6281567619272876)],
 'alaska': [('1KBB-US-WPNA', 'alaska', 1.0),
            ('1KBB-US-WMA', 'arizona', 0.6073580829582198),
            ('1KBB-US-MLG', 'michigan', 0.5603002978786962),
            ('1KBB-US-SWL', 'louisiane', 0.5415065002600895),
            ('1KBB-US-MLT', 'minnesota', 0.5338365804645864)],
 'album': [('6RF', 'rock', 0.4958178578560616),
           ('1KBB-US-SWAL', 'little rock', 0.48457967261222584),
           ('6RB', 'roman', 

In [37]:
import sys
from pprint import pprint
epsilon = sys.float_info.epsilon


def calculer_metriques_top_n(predictions_themes_top_n, donnees_references=themes_jeu_validation_dr):
    print("len(theme evaluation)", len(predictions_themes_top_n))  # contient les evaluations positives du modele
    print("len(donnees_references)", len(donnees_references)) # contient les données de références de tout le jeu de données de validation

    VP = epsilon # eviter division par 0
    VP_liste = []
    FP = 0
    FP_liste = []
    for prediction_theme_nom, prediction_theme_codes in predictions_themes_top_n.items():
        prediction_theme_codes = [code for code, nom, similarite in prediction_theme_codes]
        est_VP = False
        for prediction_theme_code in prediction_theme_codes:
            if not est_VP and prediction_theme_nom in donnees_references and prediction_theme_code in donnees_references[prediction_theme_nom]:
                VP_liste.append((prediction_theme_nom, prediction_theme_code, donnees_references[prediction_theme_nom]))
                VP += 1
                est_VP = True
        if est_VP:
            continue
        elif not est_VP and prediction_theme_nom in donnees_references and prediction_theme_code not in donnees_references[prediction_theme_nom]:
            prediction_theme_nom_thema = thema_code_dict[prediction_theme_code]["CodeDescription"] # nom du theme evalué par le modele dans théma
            jeu_validation_theme_code = donnees_references[prediction_theme_nom][0]
            if jeu_validation_theme_code != 'N/A' :
                jeu_validation_nom_theme = thema_code_dict[jeu_validation_theme_code]["CodeDescription"] # nom du theme des données de rérérence dans thema
                FP_liste.append(f"{prediction_theme_nom} -> {prediction_theme_nom_thema} vs φ={jeu_validation_nom_theme} | {prediction_theme_code} vs φ={jeu_validation_theme_code}")
            else:
                FP_liste.append(f"{prediction_theme_nom} -> {prediction_theme_nom_thema} vs φ=N/A | {prediction_theme_code} vs φ={jeu_validation_theme_code}")
            FP += 1

    FN = len(donnees_references) - len(prediction_theme_code)  # FN = N =  tout le jeu de donneés - P
    print("VP", VP)
    pprint(VP_liste)
    print("FP", FP)
    pprint(FP_liste)
    print("FN", FN)
    # les predictions du modele correspondent aux predictions positives: nous présumons que tous les mots-clés ont un thème correspondant dans théma      
    # assert(FP == len(predictions_themes_top_n) - VP) # FP = P - VP
    # assert(len(donnees_references) == VP + FN + FP)


    accuracy = (VP / len(donnees_references))
    precision = VP / (VP + FP)
    recall = VP / (VP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)

    print("VP", VP)
    print("FP", FP)

    print("accuracy", accuracy)
    print("precision", precision)
    print("rappel", recall)
    print("f1_score", f1_score)

calculer_metriques_top_n(predictions_doc2vec_top5)

len(theme evaluation) 100
len(donnees_references) 100
VP 27.0
[('sénégal', '1HFDS', ['1HFDS', '1HFDS']),
 ('alaska', '1KBB-US-WPNA', ['1KBB-US-WPNA', '1KBB-US-WPNA']),
 ('bolivie', '1KLSL', ['1KLSL', '1KLSL']),
 ('yémen', '1FBXY', ['1FBXY', '1FBXY']),
 ('père noël', '5HPD', ['5HPD', 'N/A']),
 ('sida', 'MJCJ2', ['MJCJ2', 'MJCJ2']),
 ('iran', '1FBN', ['1FBN', '1FBN']),
 ('finlande', '1DNF', ['1DNF', '1DNF']),
 ('guatemala', '1KLCG', ['1KLCG', '1KLCG']),
 ('préhistoire', '3B', ['3B', '3B']),
 ('documentaire', 'ATFR', ['YN', 'ATFR']),
 ('oman', '1FBXM', ['1FBXM', '1FBXM']),
 ('corse', '2ADH', ['1DDF-FR-I', '2ADH']),
 ('harcèlement', 'JBFK4', ['JBFK4', 'YXQF']),
 ('ouzbékistan', '1FCU', ['1FCU', '1FCU']),
 ("europe de l'est", '1DT', ['1DT', '1DT']),
 ('irak', '1FBQ', ['1FBQ', '1FBQ']),
 ('pologne', '1DTP', ['1DTP', '1DTP']),
 ('débarquement', 'JWCK', ['JWCK', '3MPBLB', 'JWCK']),
 ('aide humanitaire', 'JKSR', ['JKSR', 'JKSR']),
 ('tchécoslovaquie', '1QBDK', ['1QBDK', '1QBDK']),
 ('eau', 'WNC

## extraire adjectif, verbe et nom de la phrase

In [6]:
def extraire_adj_verbe_nom(phrase):
    doc = modele_nlp_spacy(phrase)
    tokens_filtres = [token.text for token in doc if token.pos_ in ['VERB', 'ADJ', 'NOUN'] or token.dep_ == 'ROOT'] # garder si verbe/adjectif/nom ou mot principal de la phrase
    phrase_filtree = ' '.join(tokens_filtres)
    if phrase_filtree == '':
        for token in doc:
            print(token.text, token.pos_, token.dep_)
    return phrase_filtree

# Example usage
phrase = "Théorie de l'art"
filtered_sentence = extraire_adj_verbe_nom(phrase)
print(filtered_sentence)

Théorie art


In [7]:
noeuds_thema_cache_filtre = {}
for code, theme in thema_code_dict.items():
    # Preprocess the code description
    theme_temp = {}
    nom_theme_filtre = extraire_adj_verbe_nom(theme["CodeDescription"])
    nom_theme_filtre = nom_theme_filtre.lower()
    theme_temp["CodeDescription"] = nom_theme_filtre
    theme_temp["CodeDescription_Latent"] = modele_nlp_spacy(nom_theme_filtre)
    noeuds_thema_cache_filtre[code] = theme_temp

noeuds_thema_cache_filtre

{'A': {'CodeDescription': 'arts', 'CodeDescription_Latent': arts},
 'AB': {'CodeDescription': 'arts généralités',
  'CodeDescription_Latent': arts généralités},
 'ABA': {'CodeDescription': 'théorie art',
  'CodeDescription_Latent': théorie art},
 'ABC': {'CodeDescription': 'conservation restauration entretien œuvres art',
  'CodeDescription_Latent': conservation restauration entretien œuvres art},
 'ABK': {'CodeDescription': 'contrefaçon falsification vol œuvres art',
  'CodeDescription_Latent': contrefaçon falsification vol œuvres art},
 'ABQ': {'CodeDescription': 'art aspects financiers',
  'CodeDescription_Latent': art aspects financiers},
 'AF': {'CodeDescription': 'arts formes art',
  'CodeDescription_Latent': arts formes art},
 'AFC': {'CodeDescription': 'peinture tableaux',
  'CodeDescription_Latent': peinture tableaux},
 'AFCC': {'CodeDescription': 'tableaux aquarelle pastels',
  'CodeDescription_Latent': tableaux aquarelle pastels},
 'AFCL': {'CodeDescription': 'tableaux peint

In [23]:
themes_jeu_validation_dr_filtre = {}
for theme, codes in themes_jeu_validation_dr.items():
    theme_filtre = extraire_adj_verbe_nom(theme)
    if theme_filtre == "":
        print(theme)
    themes_jeu_validation_dr_filtre[theme_filtre] = codes
# themes_jeu_validation_dr_filtre

In [24]:
predictions_doc2vec_filtre = alignement_doc2vec(noeuds_thema_cache_filtre, modele_nlp_spacy, themes_jeu_validation_dr_filtre) 

C:\Users\dre\AppData\Local\Temp\ipykernel_19260\462604653.py:12: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarite = theme_a_predire_latent.similarity(thema_valeur_modele)


balle ballon -> fléchette 0.5932092835657652
animaux domestiques -> animaux domestiques compagnie 0.9418349891439552
sénégal -> sénégal 1.0
livre - jeu -> fantasy épique - fantasy 0.6557037315005038
atelier écriture -> documentaires jeunesse art pratique dessin peinture photographie 0.755235516881218
edition -> edition commerce livre 0.7670081986126595
ours -> documentaires jeunesse ours 0.7456784084112305
théâtre contemporain -> théâtre 0.9187365837867748
maltraitance -> jeunesse questions personnelles société intimidation violence abus 0.7758347803661249
transhumanisme -> structuralisme post-structuralisme 0.8260505076725805
genre -> relatif rôles genre stéréotypés 0.6758721539989094
alaska -> alaska 1.0
biographie -> biographie autobiographie romancée 0.9245279793675946
bolivie -> bolivie 1.0
conte détourné loup cabris -> roman épouvante histoires fantôme contemporain 0.6162876045055805
aventure mystère -> histoire 0.6801745321203173
conte détourné -> roman épouvante histoires fantô

In [25]:
predictions_doc2vec_filtre

{'balle ballon': 'SXD',
 'animaux domestiques': 'WNG',
 'sénégal': '1HFDS',
 'livre - jeu': 'FMB',
 'atelier écriture': 'YNPB',
 'edition': 'KNTP1',
 'ours': 'YNNJ23',
 'théâtre contemporain': 'DD',
 'maltraitance': 'YXQ',
 'transhumanisme': 'QDHR7',
 'genre': '5YA',
 'alaska': '1KBB-US-WPNA',
 'biographie': 'FC',
 'bolivie': '1KLSL',
 'conte détourné loup cabris': 'FKM',
 'aventure mystère': 'NH',
 'conte détourné': 'FKM',
 'yémen': '1FBXY',
 'hippocampe': 'PSAN3',
 'cow - boy': '1DFG-DE-UB',
 'dragon': '1FBZB',
 'père noël': '5HPD',
 'sida': 'MJCJ2',
 'souvenir mémoire': '5HCBA',
 'pompier': 'JKSW3',
 'iran': '1FBN',
 'semaine': '5HPFH',
 'finlande': '1DNF',
 'peintre': 'AJCP',
 'relation enfant adulte': 'MKMT3',
 'papillon': 'WFBS2',
 'souffrance': 'YXED',
 'nuit': '5HRD-FI-P',
 'guatemala': '1KLCG',
 'préhistoire': '3B',
 'astronomie': 'YPMP51',
 'révolte': '3MNBJ-CA-D',
 'naissance maternité': 'MFKC1',
 'documentaire': 'AJF',
 'oman': '1FBXM',
 'clown': 'ATQL',
 'corse': '1DDF-FR-

In [48]:
VP_liste, FP_liste = calculer_metriques(predictions_doc2vec_filtre, themes_jeu_validation_dr_filtre)

len(theme evaluation) 100
len(donnees_references) 100
VP 31
[('animaux domestiques', 'WNG', ['WNG', 'WNG']),
 ('sénégal', '1HFDS', ['1HFDS', '1HFDS']),
 ('edition', 'KNTP1', ['KNTP1', 'KNTP1']),
 ('alaska', '1KBB-US-WPNA', ['1KBB-US-WPNA', '1KBB-US-WPNA']),
 ('bolivie', '1KLSL', ['1KLSL', '1KLSL']),
 ('yémen', '1FBXY', ['1FBXY', '1FBXY']),
 ('père noël', '5HPD', ['5HPD', 'N/A']),
 ('sida', 'MJCJ2', ['MJCJ2', 'MJCJ2']),
 ('iran', '1FBN', ['1FBN', '1FBN']),
 ('finlande', '1DNF', ['1DNF', '1DNF']),
 ('souffrance', 'YXED', ['YXED', 'FXK']),
 ('guatemala', '1KLCG', ['1KLCG', '1KLCG']),
 ('préhistoire', '3B', ['3B', '3B']),
 ('astronomie', 'YPMP51', ['PG', 'YPMP51']),
 ('oman', '1FBXM', ['1FBXM', '1FBXM']),
 ('corse', '1DDF-FR-I', ['1DDF-FR-I', '2ADH']),
 ('cosmonaute spationaute', 'TTDS', ['TTDS', 'TTDX', 'FLW', 'PG']),
 ('harcèlement', 'JBFK4', ['JBFK4', 'YXQF']),
 ('ouzbékistan', '1FCU', ['1FCU', '1FCU']),
 ('irak', '1FBQ', ['1FBQ', '1FBQ']),
 ('vacances', 'WT', ['WT', 'WTHF']),
 ('pologn

In [57]:
FP_liste

['balle ballon -> Fléchette vs φ=Sports / jeux de ballon | SXD vs φ=SF',
 'livre - jeu -> Fantasy épique et l’heroic-fantasy vs φ=Livres d’histoires illustrés : imagination et jeux | FMB vs φ=YBCS2',
 'atelier écriture -> Documentaires jeunesse : art pratique, dessin, peinture, photographie vs φ=Arts de l’écriture et techniques de rédaction | YNPB vs φ=CBW',
 'ours -> Documentaires jeunesse : ours vs φ=Vie sauvage : mammifères : généralités | YNNJ23 vs φ=WNCF',
 'théâtre contemporain -> Théâtre vs φ=Théâtre moderne et contemporain (à partir de 1900) | DD vs φ=DDC',
 'maltraitance -> Jeunesse : questions personnelles et de société : intimidation, violence et abus vs φ=Jeunesse : questions personnelles et de société : maltraitance | YXQ vs φ=YXQD',
 'transhumanisme -> Structuralisme et post-structuralisme vs φ=Technologies numériques et de l’information : questions d’éthique et sociales | QDHR7 vs φ=UBJ',
 'genre -> Relatif aux rôles de genre non stéréotypés vs φ=Théorie des genres | 5YA

| thème à prédire               | prédiction modèle (phrase du modèle sont filtrée, mais pas montré ici)           | prédiction humain                                                                                       | similarité cosine | prediction acceptable (sujet connexe, sans prendre en compte si ca convient aux enfants) | VJM                                                      | VJM acceptable                   |
| ----------------------------- | -------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------- | ----------------- | ---------------------------------------------------------------------------------------- | -------------------------------------------------------- | -------------------------------- |
| balle ballon                  | Fléchette                                                                        | Sports / jeux de ballon                                                                                 | 0.593             | non                                                                                      | Balle et ballon (Jeux)                                   | oui                              |  |
| livre - jeu                   | Fantasy épique et l’heroic-fantasy                                               | Livres d’histoires illustrés : imagination et jeux                                                      | 0.656             | non                                                                                      | N/A                                                      | non                              |  |
| atelier écriture              | Documentaires jeunesse : art pratique, dessin, peinture, photographie            | Arts de l’écriture et techniques de rédaction                                                           | 0.767             | non                                                                                      | N/A                                                      | non                              |  |
| ours                          | Documentaires jeunesse : ours                                                    | Vie sauvage : mammifères : généralités                                                                  | 0.746             | oui                                                                                      | winnie, ours, ours brun, ours blanc etc                  | oui                              |  |
| théâtre contemporain          | Théâtre                                                                          | Théâtre moderne et contemporain (à partir de 1900)                                                      | 0.919             | oui                                                                                      | N/A                                                      | non                              |  |
| maltraitance                  | Jeunesse : questions personnelles et de société : intimidation, violence et abus | Jeunesse : questions personnelles et de société : maltraitance                                          | 0.78              | oui                                                                                      | N/A                                                      | non                              |  |
| transhumanisme                | Structuralisme et post-structuralisme                                            | Technologies numériques et de l’information : questions d’éthique et sociales                           | 0.826             | non                                                                                      | N/A                                                      | non                              |  |
| genre                         | Relatif aux rôles de genre non stéréotypés                                       | Théorie des genres                                                                                      | 0.676             | non                                                                                      | Francais (Langue) - Genre                                | oui                              |  |
| biographie                    | Biographie/autobiographie romancée                                               | Biographie : généralités                                                                                | 0.925             | oui                                                                                      | Biographies                                              | oui                              |  |
| aventure mystère              | Histoire                                                                         | Fiction jeunesse : romans policiers et à suspense                                                       | 0.608             | non                                                                                      | N/A                                                      | non                              |  |
| conte détourné loup cabris    | Roman d’épouvante et histoires de fantôme contemporain                           | Fiction jeunesse : fantasy / fiction mythique                                                           | 0.616             | non                                                                                      | N/A                                                      | non                              |  |
| hippocampe                    | Imagerie cérébrale et neuroanatomie                                              | Vie marine et des bords de mer : généralités                                                            | 0.495             | oui                                                                                      | N/A mais Hippocampes si on rajoute un "s"                | oui si "s"                       |  |
| cow - boy                     | Bade-Wurtemberg                                                                  | Roman d’aventures : westerns                                                                            | 0.671             | non                                                                                      | Cowboys.                                                 | oui                              | interessant car le moteur de recherche rajoute un "s" automatiquement pour cette recherche<br>\-> champ "employé pour" |
| dragon                        | Tigre et Euphrate                                                                | Créatures mythiques : dragons                                                                           | 0.668             | non                                                                                      | Dragon Quest (Jeu)                                       | non                              |  |
| souvenir mémoire              | Jour du Souvenir (commémoration de l’Holocauste)                                 | Connaissance et psychologie cognitive                                                                   | 0.712             | oui                                                                                      | N/A mais Souvenir si on cherche 1 seul mot               | oui si  1 mot                    | la vmj Souvenir est associé à mémoire |
| pompier                       | Ambulance et services de secours                                                 | Services de lutte contre les incendies                                                                  | 0.306             | oui                                                                                      | sam le pompier mais Pompiers avec un "s"                 | oui si "s"                       |  |
| semaine                       | Semaine Sainte                                                                   | Eveil : temps et saisons                                                                                | 0.693             | oui                                                                                      | Semaine                                                  | oui                              |  |
| peintre                       | Photographie : art du portrait et de l’autoportrait                              | Artistes, monographies                                                                                  | 0.623             | non                                                                                      | N/A, Peintres si "s"                                     | oui si "s"                       |  |
| relation enfant adulte        | Psychothérapie : enfant et adolescent                                            | Relatif à l’enfance                                                                                     | 0.887             | oui                                                                                      | N/A, Relation d'aide si "relation"                       | non                              |  |
| papillon                      | Crochet                                                                          | Insectes (entomologie)                                                                                  | 0.585             | non                                                                                      | paola, papillons si "s"                                  | oui si "s"                       |  |
| nuit                          | Nuit polaire                                                                     | Eveil : temps et saisons                                                                                | 0.881             | oui                                                                                      | Nuit                                                     | oui                              |  |
| révolte                       | 1837 à 1838 (période des rebellions des Patriotes/de la rébellion du Bas-Canada) | Révolutions, insurrections, rébellions                                                                  | 0.721             | oui mais très specifique                                                                 | 1837 à 1838 rébellion canada, Revoltes si "s"            |
| naissance maternité           | Stérilité et fécondation                                                         | Jeunesse : questions personnelles et de société : nouveau bébé                                          | 0.714             | oui                                                                                      | N/A mais Naissance si "naissance"                        | oui si 1 mot                     |
| documentaire                  | Photoreportage et photographie documentaire                                      | Documentaire jeunesse : généralités                                                                     | 0.871             | oui                                                                                      | Documentaires                                            | oui                              | trouve le "s" tout seul |
| clown                         | Ballet                                                                           | Arts du cirque                                                                                          | 0.446             | non                                                                                      | Petit Clown (Personnage fictif : Duquennoy), clowns si s | oui si "s"                       |  |
| castor                        | Tigre et Euphrate                                                                | Zoologie : mammifères (mammologie)                                                                      | 0.557             | non                                                                                      | Castors si "s"                                           | oui si  "s"                      |  |
| hospitalisation               | Administration médicale et gestion                                               | Documentaires jeunesse : services médicaux et d’urgence                                                 | 0.824             | oui                                                                                      | N/A, meme avec hopital -> Hopitaux                       | oui si abstraction mot & pluriel |
| guerre conflit                | Prisonniers de guerre                                                            | Conflit et défense                                                                                      | 0.872             | oui mais très specifique                                                                 | Guerre si un mot                                         | oui si un mot                    |
| ogre géant                    | Tigre et Euphrate                                                                | Etres légendaires, mythiques et surnaturels, monstres et créatures                                      | 0.49              | non                                                                                      | Ogre-Doux                                                | oui si 1 mot                     |
| émotion                       | Jeunesse : émotions : compassion, empathie, gentillesse                          | Psychologie : émotions                                                                                  | 0.832             | oui mais très specifique                                                                 | colère, emotions si "s"                                  | oui si "s"                       |  |
| proverbe                      | Dialecte, argot et jargon                                                        | Dictionnaires de citations, proverbes et dictons                                                        |                   | non                                                                                      | N/A                                                      | non                              |  |
| europe                        | Europe                                                                           | Europe de l’Est                                                                                         | 1                 | oui                                                                                      | bcp de résultats -> pageRank ?                           | oui                              |  |
| coloriage livre colorier      | Livre et travail du papier                                                       | Livres animés et d’activités : dessin, coloriage, peinture                                              | 0.693             | non                                                                                      | Albums à colorier.                                      | oui                              |  |
| trésor                        | Royaume Ashanti                                                                  | Documentaires jeunesse : aventuriers et hors-la-loi                                                     | 0.427             | non                                                                                      | Chasse au trésor.                                       | oui                              |  |
| conte adaptation              | Roman d’aventures / d’action                                                     | Mythes et contes de fées                                                                                | 0.775             | non                                                                                      |                                                          |                                  |  |
| zoo                           | Médecine vétérinaire : animaux exotiques et de zoo                               | Zoos et parcs animaliers : généralités                                                                  | 0.654             | oui                                                                                      | Animaux de jardin zoologique                             | non                              |  |
| imagier                       | Dessin et dessins                                                                | N/A                                                                                                     | 0.581             | non                                                                                      |                                                          |                                  |  |
| eléphant                      | Arts                                                                             | Documentaires jeunesse : grands mammifères terrestres                                                   | 0                 | non                                                                                      | éléphants si "s"                                         | oui si "s"                       |  |
| fables                        | Contes de fées et contes populaires                                              | Fiction jeunesse : récits traditionnels                                                                 | 0.782             | oui                                                                                      | Fables                                                   |                                  |  |
| coccinelle                    | Dentelle et fabrication de dentelle                                              | Documentaires jeunesse : insectes, araignées, petites bêtes                                             | 0.556             | non                                                                                      | coccinelles                                              | oui si "s" + pageRank            |
| chauve souris                 | Oiseaux, y compris les oiseaux en cage (animaux de compagnie)                    | Vie sauvage : mammifères : généralités                                                                  | 0.533             | non                                                                                      | chauves-souris si "s" ou souris                          | oui si "s"                       |  |
| lamantin                      | Tigre et Euphrate                                                                | Vie sauvage : mammifères : généralités                                                                  | 0.506             | non                                                                                      |                                                          |                                  |  |
| écrivain écriture             | Fiction jeunesse : biographie / autobiographie romancée                          | Documentaires jeunesse : littérature, livres et écrivains                                               | 0.792             | oui                                                                                      |                                                          |                                  |  |
| histoire 18e siècle           | Poésie classique et d’avant le 20e siècle                                        | Histoire                                                                                                | 0.887             | non                                                                                      |                                                          |                                  |  |
| débarquement                  | Invasion, conquête et occupation                                                 | Forces et armées navales                                                                                | 0.715             | oui                                                                                      |                                                          |                                  |  |
| rugby                         | Rugby à quinze                                                                   | Documentaires jeunesse : jeux de balles et ballons et sports : rugby                                    | 1                 | oui mais très specifique                                                                 |                                                          |                                  |  |
| histoire texte                | Documentaires jeunesse : texte d’une pièce de théâtre                            | Livres d’histoires illustrés                                                                            | 0.778             | oui                                                                                      |                                                          |                                  |  |
| poisson                       | Cuisine par ingrédients : poisson et fruits de mer                               | Documentaires jeunesse : poissons et vie marine                                                         | 0.733             | oui                                                                                      |                                                          |                                  |  |
| lecture image                 | Traitement de l’image                                                            | N/A                                                                                                     | 0.817             | oui                                                                                      |                                                          |                                  |  |
| roller                        | Roller, skateboard, etc.                                                         | N/A                                                                                                     | 0.942             | oui                                                                                      |                                                          |                                  |  |
| licorne                       | Tigre et Euphrate                                                                | Documentaires jeunesse : licornes                                                                       | 0.563             | non                                                                                      |                                                          |                                  |  |
| regard des autres             | Asturies, principauté des vs                                                     | N/A                                                                                                     | 0.863             | non                                                                                      |                                                          |                                  |  |
| arts martiaux                 | Arts martiaux                                                                    | Documentaires jeunesse : arts martiaux                                                                  | 1                 | oui                                                                                      |                                                          |                                  |  |
| recherche origines            | Recherche scientifique                                                           | N/A                                                                                                     | 0.813             | oui                                                                                      |                                                          |                                  |  |
| eau                           | industries de l’eau                                                              | Hydrologie et hydrosphère                                                                               | 0.935             | oui                                                                                      |                                                          |                                  |  |
| londres                       | Bruxelles                                                                        | Gouvernement municipal, de la ville                                                                     | 0.642             | non                                                                                      |                                                          |                                  |  |
| conte détourné boucle or ours | Portland (OR)                                                                    | Fiction jeunesse : fantasy / fiction mythique                                                           | 0.831             | non                                                                                      |                                                          |                                  |  |
| mélancolie                    | Poésie                                                                           | Nostalgie : généralités                                                                                 | 0.656             | non                                                                                      |                                                          |                                  |  |
| tendresse baiser câlin        | Jeunesse : émotions : bonheur, joie                                              | Jeunesse : questions personnelles et de société : rencontres, relations amoureuses, romantisme et amour | 0.543             | oui                                                                                      |                                                          |                                  |  |
| dindon                        | Charolles et le Charolais                                                        | Documentaires jeunesse : oiseaux                                                                        | 0.414             | non                                                                                      |                                                          |                                  |  |
| antisémitisme                 | Anarchisme                                                                       | Jeunesse : questions personnelles et de société : préjugés & intolérance                                | 0.8               | non                                                                                      | N/A                                                      |                                  |  |
| fruit                         | Cuisine par ingrédients : poisson et fruits de mer                               | Documentaires jeunesse : cuisine et alimentation                                                        | 0.777             | non                                                                                      |                                                          |                                  |  |
| album                         | Albums pour enfant                                                               | Enfants : albums, livres d’activités, premiers apprentissages                                           | 0.63              | oui                                                                                      |                                                          |                                  |  |
| vue                           | Théorie psychologique, systèmes, écoles de pensée et points de vue               | N/A                                                                                                     | 0.6355            | non                                                                                      |                                                          |                                  |  |
| tatou                         | Tigre et Euphrate                                                                | Documentaires jeunesse : petits mammifères terrestres                                                   | 0.541             | non                                                                                      |                                                          |                                  |  |
| ennui                         | Jeunesse : émotions : inquiétude, peur                                           | Jeunesse : questions personnelles et de société : humeurs et émotions                                   | 0.437             | non                                                                                      |                                                          |                                  |  |
| végétarisme végétalisme       | Naturalisme                                                                      | Jeunesse : questions personnelles et de société : forme physique, exercice et alimentation saine        | 0.759             | non                                                                                      |                                                          |                                  |  |
| champignon                    | Protozoaire                                                                      | Mycologie                                                                                               | 0.6311            | non                                                                                      |                                                          |                                  |  |

In [27]:
predictions_doc2vec_top5_filtre = alignement_doc2vec_top_n(noeuds_thema_cache_filtre, modele_nlp_spacy, themes_jeu_validation_dr_filtre, top_n=5)

C:\Users\dre\AppData\Local\Temp\ipykernel_19260\1975866870.py:9: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarite = theme_a_predire_latent.similarity(thema_valeur_modele)


In [28]:
pprint(predictions_doc2vec_top5_filtre)

{'aide humanitaire': [('JKSR', 'aide programmes aide', 0.9300514753519773),
                      ('JBFG1', 'aide fuite évasion', 0.7851807763942561),
                      ('MRGL',
                       'aides révision médicale professional',
                       0.6890296273211581),
                      ('MRGD', 'aides révision médicale', 0.687230600114932),
                      ('MRGK',
                       'aides révision médicale surgeons mrcs',
                       0.6849356711287795)],
 'alaska': [('1KBB-US-WPNA', 'alaska', 1.0),
            ('1KBB-US-WMA', 'arizona', 0.6073580829582198),
            ('1KBB-US-MLG', 'michigan', 0.5603002978786962),
            ('1KBB-US-SWL', 'louisiane', 0.5415065002600895),
            ('1KBB-US-MLT', 'minnesota', 0.5338365804645864)],
 'album': [('YBC', 'albums enfant', 0.6303010821637851),
           ('YBCH', 'albums personnages', 0.5768743618371824),
           ('ATFN', 'film', 0.5702385305681354),
           ('AVLM', 'musique film

In [39]:
calculer_metriques_top_n(predictions_doc2vec_top5_filtre, themes_jeu_validation_dr_filtre)

len(theme evaluation) 100
len(donnees_references) 100
VP 46.0
[('animaux domestiques', 'WNG', ['WNG', 'WNG']),
 ('sénégal', '1HFDS', ['1HFDS', '1HFDS']),
 ('atelier écriture', 'CBW', ['CBW', 'CBW']),
 ('edition', 'KNTP1', ['KNTP1', 'KNTP1']),
 ('théâtre contemporain', 'DDC', ['DDC', 'DDC']),
 ('genre', 'JBSF', ['JBSF', 'YXC']),
 ('alaska', '1KBB-US-WPNA', ['1KBB-US-WPNA', '1KBB-US-WPNA']),
 ('biographie', 'YNB', ['DNB', 'YNB']),
 ('bolivie', '1KLSL', ['1KLSL', '1KLSL']),
 ('yémen', '1FBXY', ['1FBXY', '1FBXY']),
 ('dragon', 'VXQM1', ['VXQM1', 'YNXB1']),
 ('père noël', '5HPD', ['5HPD', 'N/A']),
 ('sida', 'MJCJ2', ['MJCJ2', 'MJCJ2']),
 ('iran', '1FBN', ['1FBN', '1FBN']),
 ('finlande', '1DNF', ['1DNF', '1DNF']),
 ('souffrance', 'YXED', ['YXED', 'FXK']),
 ('guatemala', '1KLCG', ['1KLCG', '1KLCG']),
 ('préhistoire', '3B', ['3B', '3B']),
 ('astronomie', 'YPMP51', ['PG', 'YPMP51']),
 ('naissance maternité', 'VFXB', ['YXFS', 'VFXB']),
 ('documentaire', 'ATFR', ['YN', 'ATFR']),
 ('oman', '1FBXM'